In [ ]:
import os
os.chdir(r'※本ファイルの保存先ディレクトリパスを指定してください')

In [ ]:
%pwd

In [ ]:
import sys
import glob
import pandas as pd
import datetime as dt
import numpy as np
import pandas as pd
import math
import re
import time
from jusho import Jusho
from logging import getLogger, FileHandler, Formatter, CRITICAL, ERROR, WARNING, INFO, DEBUG
import configparser
import shutil
import tkinter as tk
import subprocess as sp
from openpyxl import load_workbook
from tkinter import messagebox
from jusho.organize_data import create_database
from chardet.universaldetector import UniversalDetector
from openpyxl.utils.dataframe import dataframe_to_rows
from openpyxl.styles import PatternFill

cnt_err = 0
cnt_line = 0
custom_logger = ''
SAVE_LOGFILE_FOLDERPATH = ''
LOGGING_LEVEL = ''
LOGFILENAME = ''
READ_CSVFILE_FOLDERPATH = ''
STR_EXCHANGE_ADDRESS = ''
TEMPLATEFILE_FULLPATH = ''
OUTPUT_XLSXFILE_FOLDERPATH = ''

DICT_PUBLICHEALTHCARECENTER_CITY = {
    '千代田区':'千代田保健所',
    '中央区':'中央区保健所',
    '港区':'みなと保健所',
    '新宿区':'新宿区保健所',
    '文京区':'文京保健所',
    '台東区':'台東保健所',
    '墨田区':'墨田区保健所',
    '江東区':'江東区保健所',
    '品川区':'品川区保健所',
    '目黒区':'目黒区保健所',
    '大田区':'大田区保健所',
    '世田谷区':'世田谷保健所',
    '渋谷区':'渋谷区保健所',
    '中野区':'中野区保健所',
    '杉並区':'杉並保健所',
    '北区':'北区保健所',
    '豊島区':'池袋保健所',
    '荒川区':'荒川区保健所',
    '足立区':'足立保健所',
    '江戸川区':'江戸川保健所',
    '練馬区':'練馬区保健所',
    '板橋区':'板橋区保健所',
    '東村山市':'多摩小平保健所',
    '東久留米市':'多摩小平保健所',
    '清瀬市':'多摩小平保健所',
    '小平市':'多摩小平保健所',
    '西東京市':'多摩小平保健所',
    #埼玉県
    '川越市':'川越市保健所',
    '和光市':'朝霞保健所',
    '新座市':'朝霞保健所',
    '志木市':'朝霞保健所',
    '朝霞市':'朝霞保健所',
    'ふじみ野市':'朝霞保健所',
    '富士見市':'朝霞保健所',
    '入間郡三芳町':'朝霞保健所',
    'さいたま市':'さいたま市保健所',
    '川口市':'川口市保健所',
    '越谷市':'越谷市保健所',
    '蕨市':'南部保健所',
    '戸田市':'南部保健所',
    '鴻巣市':'鴻巣保健所',
    '上尾市':'鴻巣保健所',
    '桶川市':'鴻巣保健所',
    '北本市':'鴻巣保健所',
    '北足立郡伊那町':'鴻巣保健所',
    '比企郡滑川町':'東松山保健所',
    '比企郡嵐山町':'東松山保健所',
    '比企郡小川町':'東松山保健所',
    '比企郡川島町':'東松山保健所',
    '比企郡ときがわ町':'東松山保健所',
    '比企郡吉見町':'東松山保健所',
    '秩父郡東秩父村':'東松山保健所',
    '本庄市':'本庄保健所',
    '児玉郡美里町':'本庄保健所',
    '児玉郡神川町':'本庄保健所',
    '児玉郡植里町':'本庄保健所',
    '加須市':'加須保健所',
    '行田市':'加須保健所',
    '羽生市':'加須保健所',
    '春日部市':'春日部保健所',
    '北葛飾郡松伏町':'春日部保健所',
    '幸手市':'幸手保健所',
    '久喜市':'幸手保健所',
    '蓮田市':'幸手保健所',
    '白岡市':'幸手保健所',
    '南埼玉郡宮代町':'幸手保健所',
    '北葛飾郡杉戸町':'幸手保健所',
    '坂戸市':'坂戸保健所',
    '鶴ヶ島市':'坂戸保健所',
    '入間郡毛呂山町':'坂戸保健所',
    '入間郡越生町':'坂戸保健所',
    '比企郡鳩山町':'坂戸保健所',
    '草加市':'草加保健所',
    '八瀬市':'草加保健所',
    '三郷市':'草加保健所',
    '吉川市':'草加保健所',
    '所沢市':'狭山保健所',
    '狭山市':'狭山保健所',
    '飯能市':'狭山保健所',
    '入間市':'狭山保健所',
    '日高市':'狭山保健所',
}

OUTPUT_COLUMNS = ['報告年月日', 
'従事する病院・診療所の名称', 
'医師の氏名＿姓', 
'医師の氏名＿名', 
'診断（検案）した者（死体）の類型', 
'当該者氏名＿姓', 
'当該者氏名＿名', 
'当該者氏名＿姓＿フリガナ', 
'当該者氏名＿名＿フリガナ', 
'性別', 
'生年月日', 
'65歳以上',
'当該者所在地＿郵便番号', 
'当該者所在地＿都道府県', 
'当該者所在地＿市区町村', 
'当該者所在地＿丁目番地', 
'当該者所在地＿電話番号', 
'担当保健所＿都道府県', 
'担当保健所＿保健所名称', 
'当該者住所＿郵便番号', 
'当該者住所＿都道府県', 
'当該者住所＿市区町村', 
'当該者住所＿丁目番地', 
'当該者住所＿電話番号', 
'保護者氏名＿姓', 
'保護者氏名＿名', 
'保護者電話番号', 
'診断（検案）年月日', 
'検体採取日', 
'発病年月日（有症状の場合）', 
'死亡年月日（死亡者検案の場合）', 
'"新型コロナウイルスワクチン接種歴＿直近の新型コロナウイルスワクチン接種＿接種回数"', 
'"新型コロナウイルスワクチン接種歴＿直近の新型コロナウイルスワクチン接種＿接種年"', 
'"新型コロナウイルスワクチン接種歴＿直近の新型コロナウイルスワクチン接種＿接種月"', 
'"新型コロナウイルスワクチン接種歴＿直近の新型コロナウイルスワクチン接種＿接種日"', 
'"新型コロナウイルスワクチン接種歴＿直近の新型コロナウイルスワクチン接種＿ワクチン"', 
'重症化リスク因子となる疾患等の有無', 
'重症化リスク因子となる疾患等＿悪性腫瘍', 
'重症化リスク因子となる疾患等＿慢性呼吸器疾患(COPD等)', 
'重症化リスク因子となる疾患等＿慢性腎臓病', 
'重症化リスク因子となる疾患等＿心血管疾患', 
'重症化リスク因子となる疾患等＿脳血管疾患', 
'重症化リスク因子となる疾患等＿喫煙歴', 
'重症化リスク因子となる疾患等＿高血圧', 
'重症化リスク因子となる疾患等＿糖尿病', 
'重症化リスク因子となる疾患等＿脂質異常症', 
'重症化リスク因子となる疾患等＿肥満（BMI30以上）', 
'重症化リスク因子となる疾患等＿臓器の移植、免疫抑制剤、抗がん剤等の使用、その他の事由による免疫機能の低下', 
'重症化リスク因子となる疾患等＿妊娠', 
'重症化リスク因子となる疾患等＿その他', 
'治療薬の投与又は酸素投与の必要性＿治療薬の投与の必要性の有無',
'治療薬の投与又は酸素投与の必要性＿酸素投与の必要性の有無',
'発生届出時点の重症度', 
'発生届出時点の重症度＿発生届出時点の入院の必要性', 
'発生届出時点の重症度＿発生届出時点の入院の有無', 
'発生届出時点の入院＿入院年月日', 
'発生届出時点の入院＿入院医療機関', 
'その他の住所＿郵便番号', 
'その他の住所＿都道府県', 
'その他の住所＿市区町村', 
'その他の住所＿丁目番地', 
'その他の住所＿電話番号', 
'職業', 
'診断方法＿検査方法', 
'診断方法＿検体', 
'診断方法＿検査結果', 
'診断方法＿自由記述欄', 
'症状＿発熱', 
'症状＿咳', 
'症状＿咳以外の急性呼吸器症状', 
'症状＿肺炎像', 
'症状＿重篤な肺炎', 
'症状＿急性呼吸窮迫症候群', 
'症状＿多臓器不全', 
'症状＿全身倦怠感', 
'症状＿頭痛', 
'症状＿嘔気/嘔吐', 
'症状＿下痢', 
'症状＿結膜炎', 
'症状＿嗅覚・味覚障害', 
'症状＿咽頭痛', 
'症状＿酸素飽和度（室内気）', 
'症状＿その他症状', 
'ゲノム解析＿変異株PCR＿N501Y', 
'ゲノム解析＿変異株PCR＿L452R', 
'ゲノム解析＿ゲノム解析結果', 
'ゲノム解析＿ゲノム解析結果（ドロップダウン）', 
'ゲノム解析＿ゲノム解析結果（その他）', 
'ゲノム解析＿特記事項欄', 
'感染経路分析＿リンクの有無', 
'感染経路分析＿感染場所', 
'感染経路分析＿感染場所＿日本国内（都道府県）', 
'感染経路分析＿感染場所＿日本国内（市区町村）', 
'感染経路分析＿感染場所＿日本国内＿場所区分', 
'感染経路分析＿感染場所＿日本国内＿場所名称', 
'感染経路分析＿感染場所＿国外_国', 
'感染経路分析＿感染場所＿国外＿詳細地域', 
'感染経路分析＿感染場所＿国外＿渡航期間（出国日）', 
'感染経路分析＿感染場所＿国外＿渡航期間（入国日）', 
'感染経路分析＿感染原因・感染経路', 
'感染経路分析＿飛沫・飛沫核感染', 
'感染経路分析＿飛沫・飛沫核感染＿コメント', 
'感染経路分析＿接触感染', 
'感染経路分析＿接触感染＿コメント', 
'感染経路分析＿その他', 
'感染経路分析＿その他＿コメント', 
'感染経路分析＿パスポート番号', 
'感染経路分析＿医師が必要と認める事項', 
'自由記述欄（担当保健所専用）', 
'通知設定＿メール送信', 
'通知設定＿My HER-SYS URL 通知', 
'通知設定＿HER-SYS ID 通知', 
'健康観察期間']

DEFLIST_COUNTRIES = ['日本国',
'アンドラ公国',
'アラブ首長国連邦',
'アフガニスタン・イスラム共和国',
'アンティグア・バーブーダ',
'アンギラ',
'アルバニア共和国',
'アルメニア共和国',
'アンゴラ共和国',
'南極',
'アルゼンチン共和国',
'米領サモア',
'オーストリア共和国',
'オーストラリア連邦',
'アルバ',
'オーランド諸島',
'アゼルバイジャン共和国',
'ボスニア・ヘルツェゴビナ',
'バルバドス',
'バングラデシュ人民共和国',
'ベルギー王国',
'ブルキナファソ',
'ブルガリア共和国',
'バーレーン王国',
'ブルンジ共和国',
'ベナン共和国',
'サンバルテルミー',
'バミューダ諸島',
'ブルネイ・ダルサラーム国',
'ボリビア多民族国',
'オランダカリブ領域',
'ブラジル連邦共和国',
'バハマ国',
'ブータン王国',
'ブーベ島',
'ボツワナ共和国',
'ベラルーシ共和国',
'ベリーズ',
'カナダ',
'ココス諸島',
'コンゴ民主共和国',
'中央アフリカ共和国',
'コンゴ共和国',
'スイス連邦',
'コートジボワール共和国',
'クック諸島',
'チリ共和国',
'カメルーン共和国',
'中華人民共和国',
'コロンビア共和国',
'コスタリカ共和国',
'キューバ共和国',
'カーボベルデ共和国',
'キュラソー',
'クリスマス島',
'キプロス共和国',
'チェコ共和国',
'ドイツ連邦共和国',
'ジブチ共和国',
'デンマーク王国',
'ドミニカ国',
'ドミニカ共和国',
'アルジェリア民主人民共和国',
'エクアドル共和国',
'エストニア共和国',
'エジプト・アラブ共和国',
'西サハラ',
'エリトリア国',
'スペイン',
'エチオピア連邦民主共和国',
'フィンランド共和国',
'フィジー共和国',
'フォークランド（マルビナス）諸島',
'ミクロネシア連邦',
'フェロー諸島',
'フランス共和国',
'ガボン共和国',
'グレートブリテン・北アイルランド連合王国（英国）',
'グレナダ',
'ジョージア',
'フランス領ギアナ',
'ガーンジー島',
'ガーナ共和国',
'ジブラルタル',
'グリーンランド',
'ガンビア共和国',
'ギニア共和国',
'グアドループ島',
'赤道ギニア共和国',
'ギリシャ共和国',
'南ジョージア島・南サンドイッチ諸島',
'グアテマラ共和国',
'グアム',
'ギニアビサウ共和国',
'ガイアナ共和国',
'ホンコン（香港）特別行政区',
'ハード島・マクドナルド諸島',
'ホンジュラス共和国',
'クロアチア共和国',
'ハイチ共和国',
'ハンガリー',
'インドネシア共和国',
'アイルランド',
'イスラエル国',
'マン島',
'インド',
'英領インド洋地域',
'イラク共和国',
'イラン・イスラム共和国',
'アイスランド共和国',
'イタリア共和国',
'ジャージー島',
'ジャマイカ',
'ヨルダン・ハシミテ王国',
'ケニア共和国',
'キルギス共和国',
'カンボジア王国',
'キリバス共和国',
'コモロ連合',
'セントキッツ・ネービス',
'北朝鮮',
'大韓民国',
'クウェート国',
'ケイマン諸島',
'カザフスタン共和国',
'ラオス人民民主共和国',
'レバノン共和国',
'セントルシア',
'リヒテンシュタイン公国',
'スリランカ民主社会主義共和国',
'リベリア共和国',
'レソト王国',
'リトアニア共和国',
'ルクセンブルク大公国',
'ラトビア共和国',
'リビア',
'モロッコ王国',
'モナコ公国',
'モルドバ共和国',
'モンテネグロ',
'サンマルタン',
'マダガスカル共和国',
'マーシャル諸島共和国',
'北マケドニア共和国',
'マリ共和国',
'ミャンマー連邦共和国',
'モンゴル国',
'マカオ（澳門）特別行政区',
'北マリアナ諸島',
'マルチニーク島',
'モーリタニア・イスラム共和国',
'モントセラト',
'マルタ共和国',
'モーリシャス共和国',
'モルディブ共和国',
'マラウイ共和国',
'メキシコ合衆国',
'マレーシア',
'モザンビーク共和国',
'ナミビア共和国',
'ニューカレドニア',
'ニジェール共和国',
'ノーフォーク島',
'ナイジェリア連邦共和国',
'ニカラグア共和国',
'オランダ王国',
'ノルウェー王国',
'ネパール連邦民主共和国',
'ナウル共和国',
'ニウエ',
'ニュージーランド',
'オマーン国',
'パナマ共和国',
'ペルー共和国',
'フランス領ポリネシア',
'パプアニューギニア独立国',
'フィリピン共和国',
'パキスタン・イスラム共和国',
'ポーランド共和国',
'サンピエール島・ミクロン島',
'ピトケアン諸島',
'プエルトリコ',
'パレスチナ',
'ポルトガル共和国',
'パラオ共和国',
'パラグアイ共和国',
'カタール国',
'レユニオン',
'ルーマニア',
'セルビア共和国',
'ロシア連邦',
'ルワンダ共和国',
'サウジアラビア王国',
'ソロモン諸島',
'セーシェル共和国',
'スーダン共和国',
'スウェーデン王国',
'シンガポール共和国',
'セントヘレナ・アセンション・トリスタンダクーニャ',
'スロベニア共和国',
'スバールバル諸島・ヤンマイエン島',
'スロバキア共和国',
'シエラレオネ共和国',
'サンマリノ共和国',
'セネガル共和国',
'ソマリア連邦共和国',
'スリナム共和国',
'南スーダン共和国',
'サントメ・プリンシペ民主共和国',
'エルサルバドル共和国',
'シント・マールテン',
'シリア・アラブ共和国',
'エスワティニ王国',
'タークス・カイコス諸島',
'チャド共和国',
'フランス領極南諸島',
'トーゴ共和国',
'タイ王国',
'タジキスタン共和国',
'トケラウ諸島',
'東ティモール民主共和国',
'トルクメニスタン',
'チュニジア共和国',
'トンガ王国',
'トルコ共和国',
'トリニダード・トバゴ共和国',
'ツバル',
'台湾',
'タンザニア連合共和国',
'ウクライナ',
'ウガンダ共和国',
'米領小離島',
'アメリカ合衆国',
'ウルグアイ東方共和国',
'ウズベキスタン共和国',
'バチカン市国',
'セントビンセント・グレナディーン諸島',
'ベネズエラ・ボリバル共和国',
'英領バージン諸島',
'米領バージン諸島',
'ベトナム社会主義共和国',
'バヌアツ共和国',
'ワリス・フテュナ諸島',
'サモア独立国',
'イエメン共和国',
'マイヨット島',
'南アフリカ共和国',
'ザンビア共和国',
'ジンバブエ共和国',
'不明']

DEFLIST_TODOFUKEN = ['北海道',
'青森県',
'岩手県',
'宮城県',
'秋田県',
'山形県',
'福島県',
'茨城県',
'栃木県',
'群馬県',
'埼玉県',
'千葉県',
'東京都',
'神奈川県',
'新潟県',
'富山県',
'石川県',
'福井県',
'山梨県',
'長野県',
'岐阜県',
'静岡県',
'愛知県',
'三重県',
'滋賀県',
'京都府',
'大阪府',
'兵庫県',
'奈良県',
'和歌山県',
'鳥取県',
'島根県',
'岡山県',
'広島県',
'山口県',
'徳島県',
'香川県',
'愛媛県',
'高知県',
'福岡県',
'佐賀県',
'長崎県',
'熊本県',
'大分県',
'宮崎県',
'鹿児島県',
'沖縄県']

#Output Log-File
def logging_setup(save_logfile_folderpath, logfilename, logging_level):
    logger = getLogger('Make_InportXLSX_HER-SYS')
        
    if logging_level == 'logging.CRITICAL':
        log_level = CRITICAL
    elif logging_level == 'logging.ERROR':
        log_level = ERROR
    elif logging_level == 'logging.WARNING':
        log_level = WARNING
    elif logging_level == 'logging.INFO':
        log_level = INFO
    elif logging_level == 'logging.DEBUG':
        log_level = DEBUG
    else:
        log_level = WARNING
    
    logfile_fullpath = os.path.join(save_logfile_folderpath,logfilename)
    log_fh = FileHandler(logfile_fullpath)    
     # ログレベル定義
    log_fh.setLevel(log_level)
    # フォーマット定義
    log_format = Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
    log_fh.setFormatter(log_format)
    logger.addHandler(log_fh)
    return logger

def clear_logger_handlers(logger_item):
    while len(logger_item.handlers) > 0:
        handler_item = logger_item.handlers[0]
        logger_item.removeHandler(handler_item)
    return

#Check CSVFile Encoding
def check_encoding(file_path):
    detector = UniversalDetector()
    with open(file_path, mode='rb') as f:
        for binary in f:
            detector.feed(binary)
            if detector.done:
                break
    detector.close()
    return detector.result['encoding']

#exchange address to post-code
currentfolderpath = r'※Jushoモジュールをダウンロード後、address.dbを格納するdataフォルダの設置先ディレクトリパスを指定'
databasefile_fullpath = os.path.join(currentfolderpath, 'data','address.db')
postman = Jusho(database_path=databasefile_fullpath)
def exchange_address_postalcode(row_address_text):
    row_address_text = ''.join(row_address_text.split())
    row_address_text = re.sub('（本人携帯）','', row_address_text)
    row_address_text = re.sub('母　携帯　','', row_address_text).rstrip()
    #print(row_address_text)
    matchobj = re.search(r'[0-9０-９]', row_address_text)
    numtext_startposition = matchobj.start()
    
    #search_address = row_address_text[0:numtext_startposition].split('　')
    pat = '(...??[都道府県])((?:旭川|伊達|石狩|盛岡|奥州|田村|南相馬|那須塩原|東村山|武蔵村山|羽村|十日町|上越|富山|野々市|大町|蒲郡|四日市|姫路|大和郡山|廿日市|下>松|岩国|田川|大村|宮古|富良野|別府|佐伯|黒部|小諸|塩尻|玉野|周南)市|(?:余市|高市|[^市]{2,3}?)郡(?:玉村|大町|.{1,5}?)[町村]|(?:.{1,4}市)?[^町]{1,4}?区|.{1,7}?[市町村])(.+)'
    s_ptaddressinfo = re.split(pat, row_address_text)
    #print(s_ptaddressinfo)
    s_timeiaddress_text = s_ptaddressinfo[3]
    matchobj = re.search(r'[0-9０-９]', s_timeiaddress_text)
    numtext_startposition = matchobj.start()
    
    pt_address_todofuken = s_ptaddressinfo[1]
    pt_address_timei = s_timeiaddress_text[0:numtext_startposition]
    pt_address_sikuchoson = s_ptaddressinfo[2]
    pt_address_banti = s_timeiaddress_text[numtext_startposition:]
    
    result_prefecture = postman.search_prefectures(s_ptaddressinfo[1], 'kanji')[0]
    result_city = postman.search_cities(pt_address_sikuchoson, prefecture=result_prefecture)[0]
    result_address = postman.search_addresses(pt_address_timei, city=result_city)[0]   
    
    if pd.isna(result_address):
        return '※要住所確認', '※要住所確認', '※要住所確認', '※要住所確認' 
        #print('※要住所確認', '※要住所確認', '※要住所確認', '※要住所確認')
    else:
        pt_postalcode = result_address.hyphen_zip
        pt_postalcode = re.sub('-', '', pt_postalcode)
        return pt_postalcode, pt_address_todofuken, pt_address_sikuchoson + pt_address_timei, pt_address_banti   

#GET string today's YYYY/MM/DD 
def get_today_date():
    t_delta = dt.timedelta(hours=9)
    JST = dt.timezone(t_delta, 'JST')
    now = dt.datetime.now(JST)

    # YYYY/MM/DD hh:mm:ss形式に書式化
    today_date_YYYYMMDD = now.strftime('%Y/%m/%d')
    return today_date_YYYYMMDD

#選択肢検索_渡航元国
def search_country(str_country_word):
    str_match_country = [s for s in DEFLIST_COUNTRIES if str_country_word in s ]
    if len(str_match_country) == 0:
        return ''
    else:
        return str_match_country[0]

#選択肢検索_都道府県
def search_todofuken(str_todofuken_word):
    str_match_todofuken = [s for s in DEFLIST_TODOFUKEN if str_todofuken_word in s ]
    if len(str_match_todofuken) == 0:
        return ''
    else:
        return str_match_todofuken[0]

#生年月日文字列変換
def getstr_birthdate(input_strdatetime):
    birthdatetime = dt.datetime.strptime(input_strdatetime, '%Y/%m/%d %H:%M:%S')
    str_birthdatetime = birthdatetime.strftime('%Y/%m/%d')
    return str_birthdatetime

#NHO-Saitama-yahgee新様式対応_生年月日文字列変換
def nf_getstr_birthdate(input_strdate):
    birthdatetime = dt.datetime.strptime(input_strdate, '%Y%m%d')
    str_birthdatetime = birthdatetime.strftime('%Y/%m/%d')
    return str_birthdatetime

#ワクチン接種年月日文字列変換
def vaccination_year(str_year):
    str_year_mod = re.sub('R2年|R２年|令和2年|令和２年|R2.|Ｒ２.|R2．','2020',str_year)
    str_year_mod = re.sub('R3年|R３年|令和3年|令和３年|R3.|Ｒ３.|R3．','2021',str_year_mod)
    str_year_mod = re.sub('R4年|R４年|令和4年|令和４年|R4.|Ｒ４.|R4．','2022',str_year_mod)
    return str_year_mod

#ワクチン接種年月日文字列→（年、月、日）
def get_vaccinationdate(str_vaccinationdate):
    nums = re.findall(r"\d+", str_vaccinationdate)
    if len(nums) == 2 or len(nums) == 3:
        str_checkyear = nums[0]
        str_checkmonth = nums[1]
        str_checkday = '不明'
        str_vaccinationdate_day = '不明'
        if str_checkyear == '2':
            str_vaccinationdate_year = '2020'
        elif str_checkyear == '3':
            str_vaccinationdate_year = '2021'
        elif str_checkyear == '4':
            str_vaccinationdate_year = '2022'
        elif str_checkyear == '5':
            str_vaccinationdate_year = '2023'
        elif len(str_checkyear) == 4:
            str_vaccinationdate_year = str_checkyear
        else:
            str_vaccinationdate_year = '不明'

        if int(str_checkmonth) >= 1 and int(str_checkmonth) <= 12:
            str_vaccinationdate_month = str_checkmonth
        else:
            str_vaccinationdate_month = '不明'

        if len(nums) == 3:
            str_checkday = nums[2]
            if  int(str_checkday) >= 1 and int(str_checkday) < 31:
                str_vaccinationdate_day = str_checkday
            else:
                str_vaccinationdate_day = '不明'
    else:
        str_vaccinationdate_year = '不明'
        str_vaccinationdate_month = '不明'
        str_vaccinationdate_day = '不明'
    return str_vaccinationdate_year, str_vaccinationdate_month, str_vaccinationdate_day

#exchange_strings([変換前]:[変換後],...)→dict('[変換前]':'[変換後]',...)
def make_dict_exchange_address(str_beforeafter_words):
    if str_beforeafter_words != '':
        parged_str_exchange_address = re.split(',', str_beforeafter_words)
        dict_items = []
    else:
        return None
    
    for char in parged_str_exchange_address:
        dict_item = char.split(':')
        dict_items.append(dict_item)
    
    dict_exchenge_address = {k: v for (k, v) in dict_items}
    return dict_exchenge_address

def edit_xlsx_output_exchanged_data(madexlsxfilefullpath, xlsx_templatefile, check_str_list, dataplots_df):
    #deflist = '※感染原因・経路要確認,※,※管轄保健所を確認してください！'
    DEFLIST_CHECK_STRINGS_EMPHASIS_ON_CELLS = check_str_list.split(',')

    #エクセルファイル・シートを指定
    book = load_workbook(madexlsxfilefullpath)
    sheet = book['Sheet1']

    sheet_max_row = len(dataplots_df) + 1
    sheet_max_column = sheet.max_column

    for check_element in DEFLIST_CHECK_STRINGS_EMPHASIS_ON_CELLS:
        for row in sheet.iter_rows(min_row=2, max_row=sheet_max_row, min_col=1, max_col=sheet_max_column):
            for cell in row:
                if check_element in str(cell.value):
                    #該当セルを強調表示
                    cell.fill = PatternFill(fgColor=SET_BG_COLOR_CHECKED_CELL, bgColor=SET_BG_COLOR_CHECKED_CELL, fill_type = "solid")

    check_columnnum_risksevere = []
    #「重症化リスク」文言を含む列が”有"の列をセル強調表示する対象列番号リスト取得
    check_columnnum_risksevere = getList_columnnum_risksevere(xlsx_templatefile)
    #check_columnnum_risksevere_others = check_columnnum_risksevere[1] + 1
    check_columnnum_risksevere_others = check_columnnum_risksevere[1]
    for row in sheet.iter_rows(min_row=2, max_row=sheet_max_row, min_col=1, max_col=sheet_max_column):
        for cell in row:
            #診断方法のうち検査方法、検体、検査結果がNULLの場合、セル強調表示
            if cell.column in [64, 65, 66]:
                #print(cell.value)
                if cell.value == None:
                    #該当セルを強調表示
                    #print(cell.row + ',' + cell.column)
                    cell.fill = PatternFill(fgColor=SET_BG_COLOR_CHECKED_CELL,bgColor=SET_BG_COLOR_CHECKED_CELL, fill_type = "solid")
            #「重症化リスク」文言を含む列が”有"の列をセル強調表示
            if len(check_columnnum_risksevere[0]) > 0:
                if cell.column in check_columnnum_risksevere[0]:
                    if cell.value == '有':
                        #print(cell.row + ',' + cell.column)
                        cell.fill = PatternFill(fgColor=SET_BG_COLOR_CHECKED_RISKSEVERE_CELL, bgColor=SET_BG_COLOR_CHECKED_RISKSEVERE_CELL, fill_type = "solid")
            if check_columnnum_risksevere_others != 0:
                if cell.column  == check_columnnum_risksevere_others:
                    if cell.value is not None:
                        cell.fill = PatternFill(fgColor=SET_BG_COLOR_CHECKED_RISKSEVERE_CELL, bgColor=SET_BG_COLOR_CHECKED_RISKSEVERE_CELL, fill_type = "solid")
                        #custom_logger.warning('str:' + str(cell.value).lstrip().rstrip())
                        #custom_logger.warning('column:' + str(cell.column))
                        #custom_logger.warning('value:' + str(cell.value))
            

    book.save(madexlsxfilefullpath)
    
def make_xlsx_output_exchanged_data(make_filename, xlsx_templatefile, output_folderpath, dataplots_df):
    workbook = load_workbook(xlsx_templatefile)
    makexlsxfilefullpath = output_folderpath + 'dataexchangeandpasted_' + make_filename + '.xlsx'
    print(makexlsxfilefullpath)
    workbook.save(makexlsxfilefullpath)
    # 書込用に開く
    book = load_workbook(makexlsxfilefullpath)

    with pd.ExcelWriter(
        makexlsxfilefullpath,
        mode='a',
        engine="openpyxl",
        if_sheet_exists='overlay'
    ) as writer:
        dataplots_df.to_excel(writer, sheet_name='Sheet1', startrow=1, index=False, header=False, encoding='utf-8')
    edit_xlsx_output_exchanged_data(makexlsxfilefullpath, xlsx_templatefile, CHECK_STR_LIST_XLSX, dataplots_df)

def duplicate_rename(file_fullpath):
    if os.path.exists(file_fullpath):
        name, ext = os.path.splitext(file_fullpath)
        i = 1
        while True:
            # 数値を3桁などにしたい場合は({:0=3})とする
            new_name = "{} ({}){}".format(name, i, ext)
            if not os.path.exists(new_name):
                return new_name
            i += 1
    else:
        return file_fullpath
    
#phonenumber 桁数チェック
def check_phonenumberchars(str_check_phonenumber):
    ret_phonenumber = str_check_phonenumber
    if type(str_check_phonenumber) != str:
        return '※引数エラー（電話番号が文字列として設定されていません）'
    if str_check_phonenumber != '':
        check_first3chars = ['090', '080', '070']
        first_3chars_nowphonenumber = str_check_phonenumber[0:4]
        if first_3chars_nowphonenumber in check_first3chars:
            if len(str_check_phonenumber) > 11:
                ret_phonenumber = str_check_phonenumber + '※番号確認（11桁超過）'
            elif len(str_check_phonenumber) < 11:
                ret_phonenumber = str_check_phonenumber + '※番号確認（11桁未満）'
    else:
        ret_phonenumber = ''
    return ret_phonenumber
                
#xlsxテンプレートから「重症化リスク」の文言を含む列番号リストを取得する
def getList_columnnum_risksevere(path_xlsx_templatefile):
    try:
        df = pd.read_excel(path_xlsx_templatefile)
        array_bgrisksevere_columns = []
        array_bgrisksevere_columns_others = 0
        for column_no, column_item in enumerate(df.columns,1):
            if '重症化リスク' in column_item:
                if 'その他' in column_item:
                    array_bgrisksevere_columns_others = column_no
                else:
                    array_bgrisksevere_columns.append(column_no)
        return array_bgrisksevere_columns, array_bgrisksevere_columns_others
    except Exception as e:
        return -1, -1
        
#生年月日から年齢算出
def get_age(str_birthday_YYYYMMDD):
    year = int(str_birthday_YYYYMMDD[0:4])
    month = int(str_birthday_YYYYMMDD[5:7])
    day = int(str_birthday_YYYYMMDD[8:])
    today = dt.date.today()
    birthday = dt.date(year, month, day)
    return (int(today.strftime("%Y%m%d")) - int(birthday.strftime("%Y%m%d"))) // 10000
    
#GET today's datetime 
t_delta = dt.timedelta(hours=9)
JST = dt.timezone(t_delta, 'JST')
now = dt.datetime.now(JST)
todaysnowdatetime = now.strftime('%Y%m%d%H%M%S')

try:
    #init.config    
    config = configparser.ConfigParser()
    
    #init.tkinter
    root = tk.Tk()
    root.attributes('-topmost', True)
    #メッセージダイアログ表示させるための親ウインドウ非表示化
    root.withdraw()
    #メッセージダイアログモーダル表示設定
    root.lift()
    root.focus_force()
    
    #config.ini読み込み　失敗時は強制終了
    if os.path.exists('./config.ini'):
        config.read('config.ini', 'utf-8')
    else:
        messagebox.showerror('確認', 'exeと同じ場所に動作に必要なconfig.iniファイルが存在しません！\n強制終了します。')
        sys.exit('ERROR can not read config.ini')
        
    SAVE_LOGFILE_FOLDERPATH = config['LOG']['FOLDERPATH']
    LOGGING_LEVEL = config['LOG']['LOGGING_LEVEL']
    LOGFILENAME = config['LOG']['LOGFILENAME'] + todaysnowdatetime + '.log'
    READ_CSVFILE_FOLDERPATH = config['CSV']['READ_FOLDERPATH']
    MOVE_CSVFILE_FOLDERPATH = config['CSV']['MOVE_FINISHCSV_FOLDERPATH']
    STR_EXCHANGE_ADDRESS = config['DATA_CONVERSION']['EXCHANGE_ADDRESS']
    XLSXTEMPLATEFILE_FULLPATH = config['XLSX']['TEMPLATEFILE_FULLPATH']
    OUTPUT_XLSXFILE_FOLDERPATH = config['XLSX']['SAVE_XLSX_FOLDERPATH']
    CHECK_STR_LIST_XLSX = config['XLSX']['CHECK_STRINGS_EMPHASIS_ON_CELLS']
    SET_BG_COLOR_CHECKED_CELL = config['XLSX']['SET_BG_COLOR']
    SET_BG_COLOR_CHECKED_RISKSEVERE_CELL = config['XLSX']['SET_BG_RISKSEVERE_COLOR']
    WORK_DIR_PATH = config['SETTINGS']['WORK_DIR']
    
    os.chdir(WORK_DIR_PATH)
    
    
except KeyError as e:
    sys.exit('ERROR settings ouput LOG-File.Please check config.ini[SAVE_LOGFILE_FOLDERPATH, LOGGING_LEVEL, LOGFILENAME]')

except Exception as e:
    sys.exit('ERROR settings ouput LOG-File')


custom_logger = logging_setup(SAVE_LOGFILE_FOLDERPATH, LOGFILENAME, LOGGING_LEVEL)
custom_logger.info('-'*10 + '【START】' + '-'*10)

try:
    dict_exchange_address_bf_af = make_dict_exchange_address(STR_EXCHANGE_ADDRESS)
except Exception as e:
    sys.exit('ERROR make dict-exchange-address.')

try:
    #init.tkinter
    root = tk.Tk()
    root.attributes('-topmost', True)
    #メッセージダイアログ表示させるための親ウインドウ非表示化
    root.withdraw()
    #メッセージダイアログモーダル表示設定
    root.lift()
    root.focus_force()
    
    os.chdir(READ_CSVFILE_FOLDERPATH)
    csvfilename = glob.glob('*.csv')
    
    if len(csvfilename) != 1:
        if len(csvfilename) >= 2:
            messagebox.showerror('確認', 'CSV格納フォルダ内に２個以上のCSVファイルが存在します。\nCSVファイルはフォルダ内に１個の状態でexeを再実行してください！')
        elif len(csvfilename) == 0:
            messagebox.showerror('確認', 'CSV格納フォルダ内にCSVファイルが存在しません。\nCSVファイルをフォルダ内に１個の状態でexeを再実行してください！')
        custom_logger.warning('CSVファイル配置エラーのため終了')
        custom_logger.info('-'*10 + '【END】' + '-'*10)
        sys.exit('ERROR multiple csv files or does not exist in the folder')

    result_csvfile_encoding = check_encoding(csvfilename[0])
    if result_csvfile_encoding != 'CP932' and result_csvfile_encoding != 'SHIFT_JIS':
        messagebox.showerror('確認', 'CSVファイルが正しい文字コードで保存されていません。\nCSVファイルはSHIFT_JIS（メモ帳はANSI）で保存してください。\nファイルを再作成の上、exeを再実行してください！')
        custom_logger.warning('CSVファイルの文字コードが【' + result_csvfile_encoding + '】のため終了。SHIFT_JISでの保存が必要です。')
        custom_logger.info('-'*10 + '【END】' + '-'*10)
        sys.exit('ERROR csvfile-encoding is ' + result_csvfile_encoding + '. csvfile-encoding must the SHIFT_JIS.')

    org_csvfile_fullpath = READ_CSVFILE_FOLDERPATH + csvfilename[0]
###
except Exception as e:
    custom_logger.exception('変換前CSVファイルの読み込みで例外エラーが発生しました。')
    sys.exit(1)

#reading org_csvfile 
#日本語含むCSVを読み込むときは「cp932」で
#https://qiita.com/ponsuke0531/items/1dcfad48db1e33c16085
#df_get_csvdata = pd.read_csv(csvfilename[0], encoding='cp932')
df_get_csvdata = pd.read_csv(csvfilename[0], encoding='CP932')

# 変換用に、空のデータフレームを生成
input_trans_df = pd.DataFrame(index=[], columns=OUTPUT_COLUMNS)

"""
#Main
"""

result_df = input_trans_df
result_df['当該者所在地＿電話番号'] = result_df['当該者所在地＿電話番号'].astype(str)
result_df['保護者電話番号'] = result_df['保護者電話番号'].astype(str)

#1 - 報告年月日 Get Today's Date
todaysdate = get_today_date()
cnt_line = 1
cnt_err = 0
for row in df_get_csvdata.values:
    
    list_additionaldata = [[]]
    drname_sei = ''
    drname_mei = ''
    hpname = ''
    shindanruikei = ''
    ptname_sei = ''
    ptname_mei = ''
    ptkananame_sei = ''
    ptkananame_mei = ''
    gender = ''
    birthdate_YYYYMMDD = ''
    over65yearsold = ''
    business = ''
    postcode = ''
    todofuken = ''
    address1 = ''
    address2 = ''
    phonenumber = ''
    nowpostcode = ''
    nowtodofuken = ''
    nowaddress1 = ''
    nowaddress2 = ''
    nowphonenumber = ''
    guardianname_sei = ''
    guardianname_mei =''
    guardianaddress = ''
    gurdianphonenumber = ''
    hatsunetsu = ''
    seki = ''
    kyuuseikokyuuki = ''
    haienzou = ''
    zyutokuhaien = ''
    ards = ''
    tazoukifuzen = ''
    zenshinkentaikan = ''
    zutuu = ''
    ouki = ''
    geri = ''
    ketsumakuen = ''
    kyukakumikaku = ''
    spo2 = ''
    others = ''
    withoutsymptoms = ''
    kensahouhou = ''
    kentai = ''
    kentaisaishudate = ''
    result_lab = ''
    first_me_date = ''
    diagnosis_date = ''
    predate_infection = ''
    onset_date = ''
    death_date = ''
    infectionroute = ''
    himatukakukansen = ''
    desc_himatukakukansen = ''
    sesshokukansen = ''
    desc_sesshokukansen = ''
    kansenothers = ''
    desc_kansenothers = ''
    infectionarea = ''
    infection_todofuken = ''
    infection_shichouson = ''
    destinationcountry = ''
    detailedarea = ''
    departure_date = ''
    entryinto_date = ''
    vaccine_1st = ''
    vaccine_age = ''
    vaccine_type = ''
    vaccine_others = ''
    vaccine_company = ''
    vaccine_othercompany = ''
    day_vaccination_year = ''
    day_vaccination_month = ''
    day_vaccination_day = ''
    vaccine_2nd = ''
    vaccine_2nd_age = ''
    vaccine_2nd_type = ''
    vaccine_2nd_others = ''
    vaccine_2nd_company = ''
    vaccine_2nd_othercompany = ''
    day_vaccination_2ndyear = ''
    day_vaccination_2ndmonth = ''
    day_vaccination_2ndday = ''
    vaccine_3rd = ''
    vaccine_3rd_age = ''
    vaccine_3rd_type = ''
    vaccine_3rd_others = ''
    vaccine_3rd_company = ''
    vaccine_3rd_othercompany = ''
    day_vaccination_3rdyear = ''
    day_vaccination_3rdmonth = ''
    day_vaccination_3rdday = ''
    hospitalization = ''
    hospitalizationdate = ''
    severerisk = ''
    malignanttumor = ''
    copd = ''
    cdk = ''
    hbp = ''
    dm = ''
    dyslipidemia = ''
    bmi30ov = ''
    smoker = ''
    otherrisk = ''
    necessity_administration_TD = ''
    necessity_administration_O2 = ''
    immunodeficiency = ''
    pregnancy = ''
    severity = ''
    needhospitalization = ''
    telemedicine = ''
    publichealthcenter_todofuken = ''
    publichealthcenter_centername = ''
    hospitalname = ''
    periodhealthobservation = ''
    clinicaldiagnosis_comments = ''
    postcode = ''
    todofuken = ''
    publichealthcenter_todofuken = ''
    publichealthcenter_centername = ''
    address1 = ''
    address2 = ''
    nowaddressinfo = ''
    nowpostcode = ''
    nowtodofuken = ''
    nowaddress1 = ''
    nowaddress2 = ''
    cnt_person_err = 0

    custom_logger.info(str(cnt_line) + '人目：' + '-'*10 + '【START】' + '-'*10 )
    #4→2 - 従事する病院・診療所の名称 -- 固定値”独立行政法人国立病院機構埼玉病院”
    hpname = '独立行政法人国立病院機構埼玉病院'
    #2→3 - 医師（姓）from l_ptcov19data_Column3(Space区切り前半)
    drname_sei = row[2].split('　')[0]
    #3→4 - 医師（名）from l_ptcov19data_Column3(Space区切り後半)
    drname_mei = row[2].split('　')[1]
    
    new_cov19_form_hp_phone_no = row[207]
    if pd.isna(new_cov19_form_hp_phone_no) != True:
        #NHO-Saitama '22-08-12 Cov-19 NEW FORM DATA CONVERSION
        #shindanruikei, #5	診断（検案）した者（死体）の類型
        if row[287] == 1:
            shindanruikei = '患者(確定例)'
        elif row[287] == 2:
            shindanruikei = '無症状病原体保持者'
        elif row[287] == 3:
            shindanruikei = '擬似症患者'
        elif row[287] == 4:
            shindanruikei = '感染症死亡者の死体'
        elif row[287] == 5:
            shindanruikei = '感染症死亡疑い者の死体'
        #kanji-name
        """
        ptname_sei, #6	当該者氏名＿姓
        ptname_mei, #7	当該者氏名＿名
        """
        ptname_sei = row[92].split('　')[0]
        ptname_mei = row[92].split('　')[1]
        #furigana
        """
        ptkananame_sei, #8	当該者氏名＿姓＿フリガナ
        ptkananame_mei, #9	当該者氏名＿名＿フリガナ
        """
        ptkananame_sei = row[179].split('　')[0]
        ptkananame_mei = row[179].split('　')[1]
        #gender, # 10	性別
        if row[288] == 1:
            gender = '男'
        elif row[288] == 2:
            gender = '女'
        elif row[288] == 3:
            gender = 'その他'
        else:
            gender = '※入力確認'
        #birthdate_YYYYMMDD,#11	生年月日
        #raw_birthdate = str(row[208]) + str(row[209]) + str(row[210]) + str(row[211]) + str(row[212]) + str(row[213]) + str(row[214]) + str(row[215]) 
        #birthdate_YYYYMMDD = nf_getstr_birthdate(raw_birthdate)
        birthdate_YYYYMMDD = getstr_birthdate(row[37])
         #22/11/16Add 65歳以上（４類型）判定処理追加
        try:
            check_age = get_age(birthdate_YYYYMMDD)
            if check_age >= 65:
                over65yearsold = '〇'
        except Exception as e:
            custom_logger.warning(str(cnt_line) + '人目('+str(ptname_sei)+' '+str(ptname_mei) + ')：年齢判定（65歳以上か否か）に失敗しました')
            over65yearsold = '※65歳以上なら〇記載'
            cnt_person_err += 1
            cnt_err += 1
            pass
        
       
        #pt_nowaddress
        """
        nowtodofuken, #13	当該者所在地＿都道府県
        nowaddress1, #14	当該者所在地＿市区町村
        nowaddress2, #15	当該者所在地＿丁目番地
        """
         #患者住所関連処理
        if pd.isnull(row[93]) == False:
            str_exchanged_otheraddress = ''
            #置き換え単語設定チェック
            if pd.isna(dict_exchange_address_bf_af):
                for key, value in dict_exchange_address_bf_af:
                    str_exchanged_otheraddress = str(row[93]).replace(key, value)
            else:
                str_exchanged_otheraddress = row[93]


            #13→19 - 当該者住所（郵便番号） from func.exchange_address_postalcode(l_ptcov19data_Column94) ret[0]
            try:
                addressinfo = exchange_address_postalcode(str_exchanged_otheraddress)
            except AttributeError as e:
                addressinfo = ['', '', '', row[93]+str('※')]
                custom_logger.warning(str(cnt_line) + '人目('+str(ptname_sei)+ ' ' + str(ptname_mei) + ')：住所から郵便番号の検索ができませんでした。正しい住所を確認してください')
                cnt_person_err += 1
                cnt_err += 1
                pass
            #nowpostcode, #12	当該者所在地＿郵便番号
            nowpostcode = addressinfo[0]
            nowtodofuken = addressinfo[1]
            #17 担当保健所（都道府県） 
            publichealthcenter_todofuken = nowtodofuken
            nowaddress1 = addressinfo[2]
            #18 担当保健所（保健所名） 辞書より住所から管轄保健所pickup.
            publichealthcenter_centername = next((value for key, value in DICT_PUBLICHEALTHCARECENTER_CITY.items() if key in nowaddress1), '※管轄保健所を確認してください！')
            nowaddress2 = addressinfo[3]
        else:
            custom_logger.warning(str(cnt_line) + '人目('+str(ptname_sei)+ ' ' + str(ptname_mei) + ')：住所が空白です。正しい住所を確認してください')
            nowaddress2 = '※要所在地住所入力' + '※カルテ確認（患者ID：' + str(row[3]) + '）→※手修正&不要文字列削除'
            cnt_err += 1
            

        #nowphonenumber, #16	電話番号1
        if pd.isna(row[94]) == False:
            nowphonenumber = re.sub('－|-|‐|−|‒|—|–|―|ｰ|─|━|ㅡ|ـ|⁻|₋', '', str(row[94]))
            nowphonenumber = check_phonenumberchars(str(nowphonenumber))
        else:
            nowphonenumber = nowaddress2 = '※要電話番号確認→手入力→不要文字列削除' + '※カルテ確認（患者ID：' + str(row[3]) + '）'
        
        if pd.isna(row[77]) == False:
            if len(row[77].split('　')) < 1:
                if ptname_sei in row[77]:
                    guardianname_sei = ptname_sei
                    guardianname_mei = row[77].replace(ptname_sei, '').lstrip()
                else:
                    guardianname_sei = row[77] + '※'.lstrip()
                    guardianname_mei = row[77] + '※'.lstrip()
            else:
                try:
                    #guardianname_sei, #24	保護者氏名＿姓
                    if ptname_sei in row[77]:
                        guardianname_sei = ptname_sei
                    else:
                        guardianname_sei = row[77].split('　')[0] + '※'
                except Exception as e:
                    guardianname_sei = ptname_sei + '※'
                    custom_logger.warning(str(cnt_line) + '人目　' + ptname_sei + ' ' + ptname_mei + ':「保護者氏名（姓）」の入力内容を確認してください。')
                    custom_logger.exception('「保護者氏名（姓）」を変換する処理で予期せぬエラーが発生しました。')
                    pass
                try:
                    #guardianname_mei, #25	保護者氏名＿名
                    if ((guardianname_sei != '') and (guardianname_sei == ptname_sei)):
                        guardianname_mei = row[77].replace(guardianname_sei, '')
                        guardianname_mei = guardianname_mei.lstrip()
                    else:
                        guardianname_mei = row[77] + '※'
                except Exception as e:
                    guardianname_mei = row[77] + '※'
                    custom_logger.warning(str(cnt_line) + '人目　' + ptname_sei + ' ' + ptname_mei + ':「保護者氏名（名）」の入力内容を確認してください。')
                    custom_logger.exception('「保護者氏名（名）」を変換する処理で予期せぬエラーが発生しました。')
                    pass
                
            #gurdianphonenumber, #26	保護者電話番号
            gurdianphonenumber = str(row[239]).replace('.0', '') + str(row[240]).replace('.0', '') + str(row[241]).replace('.0', '') + str(row[242]).replace('.0', '') + str(row[243]).replace('.0', '') + str(row[244]).replace('.0', '') + str(row[245]).replace('.0', '') + str(row[246]).replace('.0', '') + str(row[247]).replace('.0', '') + str(row[248]).replace('.0', '') + str(row[249]).replace('.0', '') 
            gurdianphonenumber = re.sub('nan', '', gurdianphonenumber)
            if gurdianphonenumber != '':
                gurdianphonenumber = check_phonenumberchars(str(gurdianphonenumber))
                
            if guardianname_sei != '' or guardianname_mei != '':
                if gurdianphonenumber == '':
                    gurdianphonenumber = '※カルテ確認（患者ID：' + str(row[3]) + '）→電話番号手入力＆電話番号以外の文字列削除'
        #diagnosis_date #27	診断（検案）年月日
        try:
            diagnosis_date = nf_getstr_birthdate('20' + str(row[250]).replace('.0', '') + str(row[251]).replace('.0', '') + str(row[252]).replace('.0', '') + str(row[253]).replace('.0', '') + str(row[254]).replace('.0', '') + str(row[255]).replace('.0', ''))
        except Exception as e:
            custom_logger.warning(str(cnt_line) + '人目　' + ptname_sei + ' ' + ptname_mei + ':「診断（検案）年月日」の入力内容を確認してください。')
            custom_logger.exception('「診断（検案）年月日」を変換する処理で予期せぬエラーが発生しました。')
            diagnosis_date = '※診断（検案）年月日要入力' + '※カルテ確認（患者ID：' + str(row[3]) + '）→※要確認、相違時手修正&日付以外の文字列削除'
            pass
        #kentaisaishudate, #28	検体採取日
        try:
            kentaisaishudate = nf_getstr_birthdate('20' + str(row[257]).replace('.0', '') + str(row[258]).replace('.0', '') + str(row[259]).replace('.0', '') + str(row[260]).replace('.0', '') + str(row[261]).replace('.0', '') + str(row[262]).replace('.0', ''))
        except Exception as e:
            custom_logger.warning(str(cnt_line) + '人目　' + ptname_sei + ' ' + ptname_mei + ':「検体採取日」の入力内容を確認してください。')
            custom_logger.exception('「検体採取日」を変換する処理で予期せぬエラーが発生しました。')
            kentaisaishudate = '※検体採取日要入力' + '※カルテ確認（患者ID：' + str(row[3]) + '）→※要確認、相違時手修正&日付以外の文字列削除'
            pass
        #onset_date, #29	発病年月日（有症状の場合）
        try:
            onset_date = nf_getstr_birthdate('20' + str(row[263]).replace('.0', '') + str(row[264]).replace('.0', '') + str(row[265]).replace('.0', '') + str(row[266]).replace('.0', '') + str(row[267]).replace('.0', '') + str(row[268]).replace('.0', ''))
        except Exception as e:
            #log
            custom_logger.warning(str(cnt_line) + '人目　' + ptname_sei + ' ' + ptname_mei + ':「発症年月日」の入力内容を確認してください。')
            custom_logger.exception('「発症年月日」を変換する処理で予期せぬエラーが発生しました。')
            onset_date = '※発病年月日要確認・入力' + '※カルテ確認（患者ID：' + str(row[3]) + '）→※要確認、相違時手修正&日付以外の文字列削除'
            pass
        #death_date, #30	死亡年月日（死亡者検案の場合）
        try:
            if pd.isna(row[269]) == False:
                death_date = nf_getstr_birthdate('20' + str(row[269]).replace('.0', '') + str(row[270]).replace('.0', '') + str(row[271]).replace('.0', '') + str(row[272]).replace('.0', '') + str(row[273]).replace('.0', '') + str(row[274]).replace('.0', ''))
            else:
                death_date = ''
        except Exception as e:
            #log
            custom_logger.warning(str(cnt_line) + '人目　' + ptname_sei + ' ' + ptname_mei + ':「死亡年月日」の入力内容を確認してください。')
            custom_logger.exception('「死亡年月日」を変換する処理で予期せぬエラーが発生しました。')
            death_date = '※死亡年月日要確認・入力' + '※カルテ確認（患者ID：' + str(row[3]) + '）→※要確認、相違時手修正&日付以外の文字列削除'
            pass
        #count_vaccines, #31	"新型コロナウイルスワクチン接種歴＿直近の新型コロナウイルスワクチン接種＿接種回数"
        count_vaccines = str(row[289]).replace('.0', '').replace('nan', '')
        if count_vaccines == '':
            count_vaccines = '不明'
        
        
        #day_vaccination_year, #32	"新型コロナウイルスワクチン接種歴＿直近の新型コロナウイルスワクチン接種＿接種年"
        #day_vaccination_month, #33	"新型コロナウイルスワクチン接種歴＿直近の新型コロナウイルスワクチン接種＿接種月"
        #day_vaccination_day, #34	"新型コロナウイルスワクチン接種歴＿直近の新型コロナウイルスワクチン接種＿接種日"
        item_vaccines_year = str(row[316]).replace('.0', '') + str(row[317]).replace('.0', '')
        item_vaccines_month = str(row[318]).replace('.0', '') + str(row[319]).replace('.0', '')
        item_vaccines_day = str(row[320]).replace('.0', '') + str(row[321]).replace('.0', '')
        if item_vaccines_year != '不明':
            if str.isdigit(item_vaccines_year):
                day_vaccination_year = int('20' + item_vaccines_year)
            else:
                day_vaccination_year = ''
        else:
            day_vaccination_year = '不明'
        
        if item_vaccines_month != '不明':
            if str.isdigit(item_vaccines_month):
                day_vaccination_month = int(item_vaccines_month)
            else:
                day_vaccination_month = '不明'
        else:
            day_vaccination_month = '不明'
            
        if item_vaccines_day != '不明':
            if str.isdigit(item_vaccines_day):
                day_vaccination_day = int(item_vaccines_day)
            else:
                day_vaccination_day = '不明'
        else:
            day_vaccination_day = '不明'
        if count_vaccines == '0':
            day_vaccination_year = ''
            day_vaccination_month = ''
            day_vaccination_day = ''
            
        if count_vaccines != '不明':
            count_vaccines = int(count_vaccines)
        
        
        #vaccine_company, #35	"新型コロナウイルスワクチン接種歴＿直近の新型コロナウイルスワクチン接種＿ワクチン"
        if pd.isna(row[290]) == False:
            vaccine_company = int(row[290])
        else:
            vaccine_company = ''
            
        if vaccine_company == 1:
            vaccine_company = 'ファイザー'
        elif vaccine_company == 2:
            vaccine_company = 'モデルナ'
        elif vaccine_company == 3:
            vaccine_company = 'アストラゼネカ'
        elif vaccine_company == 4:
            vaccine_company = 'ノババックス'
        else:
            if count_vaccines != 0:
                vaccine_company = '不明'
            else:
                vaccine_company = ''
        
        #vaccine_other
        vaccine_other = row[293]
        
        #malignanttumor, #37	重症化リスク因子となる疾患等＿悪性腫瘍
        check_malignanttumor = lambda x : '有' if x == 1 else '無'
        malignanttumor = check_malignanttumor(row[157])
        #copd, #38	重症化リスク因子となる疾患等＿慢性呼吸器疾患(COPD等)
        check_copd = lambda x : '有' if x == 1 else '無'
        copd = check_copd(row[158])
        #cdk, #39	重症化リスク因子となる疾患等＿慢性腎臓病
        check_cdk = lambda x : '有' if x == 1 else '無'
        cdk = check_cdk(row[159])
        #cardiovascular_disease, #40	重症化リスク因子となる疾患等＿心血管疾患
        check_cardiovascular_disease = lambda x : '有' if x == 1 else '無'
        cardiovascular_disease = check_cardiovascular_disease(row[294])
        #cerebrovascular_disease, #41	重症化リスク因子となる疾患等＿脳血管疾患
        check_cerebrovascular_disease = lambda x : '有' if x == 1 else '無'
        cerebrovascular_disease = check_cerebrovascular_disease(row[295])
        #smoker, #42	重症化リスク因子となる疾患等＿喫煙歴
        check_smoker = lambda x : '有' if x == 1 else '無'
        smoker = check_smoker(row[164])
        #hbp, #43	重症化リスク因子となる疾患等＿高血圧
        check_hbp = lambda x : '有' if x == 1 else '無'
        hbp = check_hbp(row[160])
        #dm, #44	重症化リスク因子となる疾患等＿糖尿病
        check_dm = lambda x : '有' if x == 1 else '無'
        dm = check_dm(row[161])
        #dyslipidemia, #45	重症化リスク因子となる疾患等＿脂質異常症
        check_dyslipidemia = lambda x : '有' if x == 1 else '無'
        dyslipidemia = check_dyslipidemia(row[162])
        #bmi30ov, #46	重症化リスク因子となる疾患等＿肥満（BMI30以上）
        check_bmi30ov = lambda x : '有' if x == 1 else '無'
        bmi30ov = check_bmi30ov(row[163])
        #immunodeficiency, #47	重症化リスク因子となる疾患等＿臓器の移植、免疫抑制剤、抗がん剤等の使用、その他の事由による免疫機能の低下
        #check_immunodeficiency = lambda x, y : '有' if x == 1 else '無' if y == 1 else '無'
        check_immunodeficiency = lambda x : '有' if x == 1 else '無'
        immunodeficiency = check_immunodeficiency(row[296])
        #pregnancy, #48	重症化リスク因子となる疾患等＿妊娠
        #check_pregnancy = lambda x, y : '有' if x == 1 else '無' if y == 1 else '無'
        check_pregnancy = lambda x : '有' if x == 1 else '無'
        pregnancy = check_pregnancy(row[297])
        #otherrisk, #49	重症化リスク因子となる疾患等＿その他
        #check_otherrisk = lambda x : '有' if x == 1 else '無'
        otherrisk = str(row[167]).replace('0.0','')
        if otherrisk == 'nan':
            otherrisk = ''
        if otherrisk == '':
            otherrisk = '無'
            necessity_administration_TD = '無'
            necessity_administration_O2 = '無'
        #22/11/16Add 治療薬の投与の必要性の有無＆酸素投与の必要性の有無判定処理追加
        elif otherrisk.find('0') == -1 or otherrisk.find('０') == -1:
            otherrisk = '無'
            necessity_administration_TD = '無'
            necessity_administration_O2 = '無'
        else:
            otherrisk = '有'
            necessity_administration_TD = '有'
            necessity_administration_O2 = '有'
            
        
        #severerisk, #36	重症化リスク因子となる疾患等の有無
        check_severerisk = any([
            malignanttumor == '有',
            copd == '有',
            cdk == '有',
            cardiovascular_disease == '有',
            cerebrovascular_disease == '有',
            smoker == '有',
            hbp == '有',
            dm == '有',
            dyslipidemia == '有',
            bmi30ov == '有',
            immunodeficiency == '有',
            pregnancy == '有',
            len(otherrisk) > 0
        ])
        if check_severerisk == True:
            severerisk = '有'
        else:
            severerisk = '無'
        
        severity = int(row[298])    
        #severity, #50	発生届出時点の重症度
        if severity == 1:
            severity = '軽症'
        elif severity == 2:
            severity = '中等症I（呼吸不全なし）'
        elif severity == 3:
            severity = '中等症II（呼吸不全あり）'
        elif severity == 4:
            severity = '重症'
        elif severity == 5:
            severity = '無症状'
        else:
            severity = '※重症度要入力'
            
        #needhospitalization, #51	発生届出時点の重症度＿発生届出時点の入院の必要性
        check_needhospitalization = lambda x : '有' if x == 1 else '無'
        needhospitalization = check_needhospitalization(row[291])
        #hospitalization, #52	発生届出時点の重症度＿発生届出時点の入院の有無
        check_hospitalization = lambda x : '有' if x == 1 else '無'
        hospitalization = check_hospitalization(row[292])
        if hospitalization == '有':
            hospitalname = '独立行政法人　国立病院機構　埼玉病院'
            date_hospitalization = diagnosis_date + '※カルテ確認（患者ID：' + str(row[3]) + '）→※要確認、相違時手修正&日付以外の文字列削除'
            
        else:
            hospitalname = ''
            date_hospitalization = ''
        
        ###臨床診断対応 - 2022.8.16 Add
        ###無症状　かつ　類型：疑似症　→　臨床診断
        if shindanruikei == '擬似症患者' and severity == '無症状':
            #63	診断方法＿検査結果
            result_lab = 'その他'
            #64	診断方法＿自由記述欄
            clinicaldiagnosis_comments = '臨床診断'
            #発症日
            onset_date = ''
        else:
            result_lab = ''
            clinicaldiagnosis_comments = ''
        
        ###症状記載なし対応　2022.8.18　Add
        symptoms_others = 'HER-SYS仕様により「症状なし」となりますが、症状はすべて「未記載」として解釈ください。本運用は厚生労働省新型コロナウイルス感染症対策推進本部戦略班に確認、了承済であります。'
        
        ####ここから行追加処理
        list_additionaldata = [[
            todaysdate, #1	報告年月日
            hpname, #2	従事する病院・診療所の名称
            drname_sei, #3	医師の氏名＿姓
            drname_mei,#4	医師の氏名＿名
            shindanruikei, #5	診断（検案）した者（死体）の類型
            ptname_sei, #6	当該者氏名＿姓
            ptname_mei, #7	当該者氏名＿名
            ptkananame_sei, #8	当該者氏名＿姓＿フリガナ
            ptkananame_mei, #9	当該者氏名＿名＿フリガナ
            gender, # 10	性別
            birthdate_YYYYMMDD,#11	生年月日
            over65yearsold,#22/11/16Add 65歳以上
            nowpostcode, #12	当該者所在地＿郵便番号
            nowtodofuken, #13	当該者所在地＿都道府県
            nowaddress1, #14	当該者所在地＿市区町村
            nowaddress2, #15	当該者所在地＿丁目番地
            nowphonenumber, #16	電話番号1
            publichealthcenter_todofuken, #17	担当保健所＿都道府県
            publichealthcenter_centername, #18	担当保健所＿保健所名称
            '', #19	当該者住所＿郵便番号
            '', #20	当該者住所＿都道府県
            '', #21	当該者住所＿市区町村
            '', #22	当該者住所＿丁目番地
            '', #23	電話番号2
            guardianname_sei, #24	保護者氏名＿姓
            guardianname_mei, #25	保護者氏名＿名
            gurdianphonenumber, #26	保護者電話番号
            diagnosis_date, #27	診断（検案）年月日
            kentaisaishudate, #28	検体採取日
            onset_date, #29	発病年月日（有症状の場合）
            death_date, #30	死亡年月日（死亡者検案の場合）
            count_vaccines, #31	"新型コロナウイルスワクチン接種歴＿直近の新型コロナウイルスワクチン接種＿接種回数"
            day_vaccination_year, #32	"新型コロナウイルスワクチン接種歴＿直近の新型コロナウイルスワクチン接種＿接種年"
            day_vaccination_month, #33	"新型コロナウイルスワクチン接種歴＿直近の新型コロナウイルスワクチン接種＿接種月"
            day_vaccination_day, #34	"新型コロナウイルスワクチン接種歴＿直近の新型コロナウイルスワクチン接種＿接種日"
            vaccine_company, #35	"新型コロナウイルスワクチン接種歴＿直近の新型コロナウイルスワクチン接種＿ワクチン"
            severerisk, #36	重症化リスク因子となる疾患等の有無
            malignanttumor, #37	重症化リスク因子となる疾患等＿悪性腫瘍
            copd, #38	重症化リスク因子となる疾患等＿慢性呼吸器疾患(COPD等)
            cdk, #39	重症化リスク因子となる疾患等＿慢性腎臓病
            cardiovascular_disease, #40	重症化リスク因子となる疾患等＿心血管疾患
            cerebrovascular_disease, #41	重症化リスク因子となる疾患等＿脳血管疾患
            smoker, #42	重症化リスク因子となる疾患等＿喫煙歴
            hbp, #43	重症化リスク因子となる疾患等＿高血圧
            dm, #44	重症化リスク因子となる疾患等＿糖尿病
            dyslipidemia, #45	重症化リスク因子となる疾患等＿脂質異常症
            bmi30ov, #46	重症化リスク因子となる疾患等＿肥満（BMI30以上）
            immunodeficiency, #47	重症化リスク因子となる疾患等＿臓器の移植、免疫抑制剤、抗がん剤等の使用、その他の事由による免疫機能の低下
            pregnancy, #48	重症化リスク因子となる疾患等＿妊娠
            otherrisk, #49	重症化リスク因子となる疾患等＿その他
            necessity_administration_TD, #22/11/16Add 治療薬の投与の必要性の有無
            necessity_administration_O2, #22/11/16Add 酸素投与の必要性の有無
            severity, #50	発生届出時点の重症度
            needhospitalization, #51	発生届出時点の重症度＿発生届出時点の入院の必要性
            hospitalization, #52	発生届出時点の重症度＿発生届出時点の入院の有無
            date_hospitalization, #53	発生届出時点の入院＿入院年月日
            hospitalname, #54	発生届出時点の入院＿入院医療機関
            '', #55	その他の住所＿郵便番号
            '', #56	その他の住所＿都道府県
            '', #57	その他の住所＿市区町村
            '', #58	その他の住所＿丁目番地
            '', #59	電話番号3
            '', #60	職業
            '', #61	診断方法＿検査方法
            '', #62	診断方法＿検体
            result_lab, #63	診断方法＿検査結果
            clinicaldiagnosis_comments, #64	診断方法＿自由記述欄
            '', #65	症状＿発熱
            '', #66	症状＿咳
            '', #67	症状＿咳以外の急性呼吸器症状
            '', #68	症状＿肺炎像
            '', #69	症状＿重篤な肺炎
            '', #70	症状＿急性呼吸窮迫症候群
            '', #71	症状＿多臓器不全
            '', #72	症状＿全身倦怠感
            '', #73	症状＿頭痛
            '', #74	症状＿嘔気/嘔吐
            '', #75	症状＿下痢
            '', #76	症状＿結膜炎
            '', #77	症状＿嗅覚・味覚障害
            '', #78	症状＿咽頭痛
            '', #79	症状＿酸素飽和度（室内気）
            symptoms_others, #80	症状＿その他症状
            '未実施', #81	ゲノム解析＿変異株PCR＿N501Y
            '未実施', #82	ゲノム解析＿変異株PCR＿L452R
            '', #83	ゲノム解析＿ゲノム解析結果
            '', #84	ゲノム解析＿ゲノム解析結果（ドロップダウン）
            '', #85	ゲノム解析＿ゲノム解析結果（その他）
            '', #86	ゲノム解析＿特記事項欄
            '', #87	感染経路分析＿リンクの有無
            '', #95	感染経路分析＿感染場所
            '', #96	感染経路分析＿感染場所＿日本国内（都道府県）
            '', #97	感染経路分析＿感染場所＿日本国内（市区町村）
            '', #98	感染経路分析＿感染場所＿日本国内＿場所区分
            '', #99	感染経路分析＿感染場所＿日本国内＿場所名称
            '', #100	感染経路分析＿感染場所＿国外_国
            '', #101	感染経路分析＿感染場所＿国外＿詳細地域
            '', #102	感染経路分析＿感染場所＿国外＿渡航期間（出国日）
            '', #103	感染経路分析＿感染場所＿国外＿渡航期間（入国日）
            '', #88	感染経路分析＿感染原因・感染経路
            '', #89	感染経路分析＿飛沫・飛沫核感染
            '', #90	感染経路分析＿飛沫・飛沫核感染＿コメント
            '', #91	感染経路分析＿接触感染
            '', #92	感染経路分析＿接触感染＿コメント
            '', #93	感染経路分析＿その他
            '', #94	感染経路分析＿その他＿コメント
            '', #104	感染経路分析＿パスポート番号
            '', #105	感染経路分析＿医師が必要と認める事項
            '', #106	自由記述欄（担当保健所専用）
            '無', #107	通知設定＿メール送信
            '無',#108	通知設定＿My HER-SYS URL 通知
            '無',#109	通知設定＿HER-SYS ID 通知
            0#110	健康観察期間
            ]]
    #NHO-Saitama old-form(before.2022.8.12)
    else:

        """
        5 - 診断（検案）した者（死体）の類型 -- from l_ptcov19data_Column15(患者(確定例))/
                                                                       96(無症状病原体保持者)/
                                                                       97(擬似症患者)/
                                                                       16(感染症死亡者の死体)/
                                                                       98(感染症死亡疑い者の死体)
        """
        if row[14] == 1:
            shindanruikei = '患者(確定例)'
        elif row[95] == 1:
            shindanruikei = '無症状病原体保持者'
        elif row[96] == 1:
            shindanruikei = '擬似症患者'
        elif row[15] == 1:
            shindanruikei = '感染症死亡者の死体'
        elif row[97] == 1:
            shindanruikei = '感染症死亡疑い者の死体'
            
        #6 - 当該者氏名（姓）from l_ptcov19data_Column93(Space区切り前半）
        ptname_sei = row[92].split('　')[0]
        #7 - 当該者氏名（名）from l_ptcov19data_Column93(Space区切り後半）
        ptname_mei = row[92].split('　')[1]
        #8 - 当該者氏名フリガナ（姓）from l_ptcov19data_Column180(Space区切り前半）
        ptkananame_sei = row[179].split('　')[0]
        #9 - 当該者氏名フリガナ（名）from l_ptcov19data_Column180(Space区切り後半）
        ptkananame_mei = row[179].split('　')[1]
        #10 - 性別 from l_ptcov19data_Column37
        gender = row[36]
        #11 - 生年月日 from l_ptcov19data_Column38 --[yyyy/mm/dd]形式に要変換
        try:
            birthdate_YYYYMMDD = getstr_birthdate(row[37])
        except Exception as e:
            custom_logger.warning(str(cnt_line) + '人目('+str(ptname_sei)+' '+str(ptname_mei) + ')：生年月日を確認してください')
            birthdate_YYYYMMDD = '※生年月日要確認'
            cnt_person_err += 1
            cnt_err += 1
            pass
        
        #22/11/16Add 65歳以上（４類型）判定処理追加
        try:
            check_age = get_age(birthdate_YYYYMMDD)
            if check_age >= 65:
                over65yearsold = '〇'
        except Exception as e:
            custom_logger.warning(str(cnt_line) + '人目('+str(ptname_sei)+' '+str(ptname_mei) + ')：年齢判定（65歳以上か否か）に失敗しました')
            birthdate_YYYYMMDD = '※65歳以上なら〇記載'
            cnt_person_err += 1
            cnt_err += 1
            pass

        if pd.isnull(row[73]) == False:
            try:
                str_exchanged_nowaddress = ''
                #置き換え単語設定チェック
                if pd.isna(dict_exchange_address_bf_af):
                    for key, value in dict_exchange_address_bf_af:
                        str_exchanged_nowaddress = str(row[73]).replace(key, value)
                else:
                    str_exchanged_nowaddress = row[73]

                nowaddressinfo = exchange_address_postalcode(str_exchanged_nowaddress)
                #18→12 - 当該者所在地（郵便番号）from func.exchange_address_postalcode(l_ptcov19data_Column74) ret[0]
                nowpostcode = nowaddressinfo[0]
                #19→13 - 当該者所在地（都道府県）from func.exchange_address_postalcode(l_ptcov19data_Column74) ret[1]
                nowtodofuken = nowaddressinfo[1]
                #20→14 - 当該者所在地（住所1）from func.exchange_address_postalcode(l_ptcov19data_Column74) ret[2]
                nowaddress1 = nowaddressinfo[2]
                #21→15 - 当該者所在地（住所2）from func.exchange_address_postalcode(l_ptcov19data_Column74) ret[3]
                nowaddress2 = nowaddressinfo[3]
            except Exception as e:
                custom_logger.warning(str(cnt_line) + '人目('+str(ptname_sei)+' '+str(ptname_mei) + ')：所在地住所を確認してください')
                custom_logger.exception('「現在所在地」の変換処理で予期せぬエラーが発生しました。')
                cnt_person_err += 1
                cnt_err += 1
                nowaddressinfo = ''
                nowpostcode = ''
                nowtodofuken = ''
                nowaddress1 = ''
                nowaddress2 = row[73] + '※住所を確認し、正しい郵便番号等手入力を行ってください！'
                pass
        else:
            nowaddressinfo = ''
            nowpostcode = ''
            nowtodofuken = ''
            nowaddress1 = ''
            nowaddress2 = ''

        nowphonenumber_item1 = ('' if pd.isna(row[74]) else str(row[74]).replace('.0', ''))
        nowphonenumber_item2 = ('' if pd.isna(row[75]) else str(row[75]).replace('.0', ''))
        nowphonenumber_item3 = ('' if pd.isna(row[76]) else str(row[76]).replace('.0', ''))
        nowphonenumber_item = nowphonenumber_item1 + nowphonenumber_item2 + nowphonenumber_item3
        #nowphonenumber_item = str('{}{}{}'.format(row[74], row[75], row[76]))
        #22→16 - 当該者所在地（電話番号）from l_ptcov19data_Column75 & 76 & 77 
        nowphonenumber = re.sub('－|-|‐|−|‒|—|–|―|ｰ|─|━|ㅡ|ـ|⁻|₋', '', nowphonenumber_item)
        if nowphonenumber_item != '':
            nowphonenumber = str('0' + nowphonenumber)
        else:
            nowphonenumber = ''

        #126→17 担当保健所（都道府県） 固定値　""埼玉県""
        #127→18 担当保健所（保健所名） 固定値　""朝霞保健所""
        #患者住所関連処理
        if pd.isnull(row[93]) == False:
            str_exchanged_otheraddress = ''
            #置き換え単語設定チェック
            if pd.isna(dict_exchange_address_bf_af):
                for key, value in dict_exchange_address_bf_af:
                    str_exchanged_otheraddress = str(row[93]).replace(key, value)
            else:
                str_exchanged_otheraddress = row[93]


            #13→19 - 当該者住所（郵便番号） from func.exchange_address_postalcode(l_ptcov19data_Column94) ret[0]
            try:
                addressinfo = exchange_address_postalcode(str_exchanged_otheraddress)
            except AttributeError as e:
                addressinfo = ['', '', '', row[93]+str('※')]
                custom_logger.warning(str(cnt_line) + '人目('+str(ptname_sei)+ ' ' + str(ptname_mei) + ')：住所から郵便番号の検索ができませんでした。正しい住所を確認してください')
                cnt_person_err += 1
                cnt_err += 1
                pass

            postcode = addressinfo[0]
            #14→20 - 当該者住所（都道府県） from func.exchange_address_postalcode(l_ptcov19data_Column94) ret[1]
            todofuken = addressinfo[1]
            #126→17 担当保健所（都道府県） 
            publichealthcenter_todofuken = todofuken
            #15→21 - 当該者住所（住所1） from func.exchange_address_postalcode(l_ptcov19data_Column94) ret[2]
            address1 = addressinfo[2]
            #127→18 担当保健所（保健所名） 辞書より住所から管轄保健所pickup.
            publichealthcenter_centername = next((value for key, value in DICT_PUBLICHEALTHCARECENTER_CITY.items() if key in address1), '※管轄保健所を確認してください！')
            #16→22 - 当該者住所（住所2） from func.exchange_address_postalcode(l_ptcov19data_Column94) ret[3]
            address2 = addressinfo[3]       

        #17→23 - 当該者住所（電話番号）from l_ptcov19data_Column95 --"ー"(ハイフン)除去
        phonenumber = re.sub('－|-|‐|−|‒|—|–|―|ｰ|─|━|ㅡ|ـ|⁻|₋', '', str(row[94]))
        phonenumber =  str(phonenumber)

        if pd.isnull(row[77]) == False:
            #23→24 - 保護者氏名（姓）from l_ptcov19data_Column78(Space区切り前半）
            guardianname_sei = row[77].split('　')[0]
            #24→25 - 保護者氏名（名）from l_ptcov19data_Column78(Space区切り後半）
            guardianname_mei = row[77].split('　')[1]
            #55～58取得か←25 - 保護者住所（住所）from l_ptcov19data_Column79
            #guardianaddress = row[78]
            #26→26 - 保護者住所（電話）from l_ptcov19data_Column81
            gurdianphonenumber_item1 = ('' if pd.isna(row[79]) else str(row[79]).replace('.0', ''))
            gurdianphonenumber_item2 = ('' if pd.isna(row[80]) else str(row[80]).replace('.0', ''))
            gurdianphonenumber_item3 = ('' if pd.isna(row[81]) else str(row[81]).replace('.0', ''))
            gurdianphonenumber_item = gurdianphonenumber_item1 + gurdianphonenumber_item2 + gurdianphonenumber_item3
            gurdianphonenumber = re.sub('－|-|‐|−|‒|—|–|―|ｰ|─|━|ㅡ|ـ|⁻|₋', '', gurdianphonenumber_item)
            if gurdianphonenumber_item != '':
                gurdianphonenumber = str('0' + gurdianphonenumber)
            else:
                gurdianphonenumber = ''

        #27←61 診断（検案）年月日 from l_ptcov19data_Column83 ※要形成変換YYYY/M/D→YYYY/MM/DD
        diagnosis_date = getstr_birthdate(row[82])

        conv_dateYYYYMMDD = lambda x : dt.datetime.strptime(x, '%Y/%m/%d').strftime('%Y/%m/%d')
        #29←63 発病年月日 from l_ptcov19data_Column85 ※要形成変換YYYY/M/D→YYYY/MM/DD
        try:
            if row[84] == '不詳' or row[84] == '不明':
                onset_date = ''
            elif pd.isnull(row[84]) == False:
                onset_date = conv_dateYYYYMMDD(row[84])
        except Exception as e:
            custom_logger.warning(str(cnt_line) + '人目　' + ptname_sei + ' ' + ptname_mei + ':「発症年月日」の入力内容を確認してください。')
            custom_logger.exception('「発症年月日」の変換処理で予期せぬエラーが発生しました。')
            onset_date = '※原本要確認、要入力'
            pass

        #30←64 死亡年月日 from l_ptcov19data_Column84 ※要形成変換YYYY/M/D→YYYY/MM/DD#64 死亡年月日 from l_ptcov19data_Column84 ※要形成変換YYYY/M/D→YYYY/MM/DD
        try:
            if math.isnan(row[83]) == False:
                death_date = conv_dateYYYYMMDD(row[83])
        except Exception as e:
            cnt_person_err += 1
            cnt_err += 1
            custom_logger.warning(str(cnt_line) + '人目　' + ptname_sei + ' ' + ptname_mei + ':「死亡年月日」の入力内容を確認してください。')
            custom_logger.exception('「死亡年月日」の変換処理で予期せぬエラーが発生しました。')
            death_date = '※原本要確認、要入力'
            pass

        count_vaccines =  0
        try:
            #新型コロナウイルスワクチン接種の有無（1回目） from l_ptcov19data_Column138(有)/140(無)/141(不明) 
            check_vaccine = lambda x, y, z : '有' if x == 1 else '無' if y == 1 else '不明' if z == 1 else ''
            vaccine_1st = check_vaccine(row[137], row[139], row[140])

            #90 新型コロナウイルスワクチン接種の有無（2回目） from l_ptcov19data_Column147(有)/149(無)/150(不明) 
            vaccine_2nd = check_vaccine(row[146], row[148], row[149])

            #99 新型コロナウイルスワクチン接種の有無（3回目） from l_ptcov19data_Column182(有)/184(無)/185(不明) 
            vaccine_3rd = check_vaccine(row[181], row[183], row[184])

            if vaccine_1st == '有':
                count_vaccines += 1

                #新型コロナウイルスワクチン接種時の年齢（1回目） from l_ptcov19data_Column139 ※138(有)のみ
                #vaccine_1st_age = row[138]
                #83 ワクチンの種類（1回目） from l_ptcov19data_Column142 mRNA/ウイルスベクター/不明/その他
                vaccine_type = row[141] if pd.isnull(row[141]) == False else '不明'
                #84 ワクチンの種類（その他）（1回目） from l_ptcov19data_Column※142”その他” その他
                if vaccine_type == 'その他':
                    vaccine_others = 'その他'
                    #86 製造会社（その他）（1回目） from l_ptcov19data_Column※142”その他” その他
                    vaccine_company = 'その他'
                else:
                    #85 製造会社（1回目） from l_ptcov19data_Column143 ファイザー/モデルナ/アストラゼネカ/ノババックス/不明/その他
                    vaccine_company = row[142] if pd.isnull(row[142]) == False else '不明'

                check_1stdate = lambda x :'不明' if pd.isnull(x) else x
                str_vaccinationdate = get_vaccinationdate(check_1stdate(row[144]))
                if row[145] == 1:
                    #87 接種年（1回目） from l_ptcov19data_Column145(※右記詳細参照)/146(不明) YYYY/M/D→YYYY/MM/DD
                    day_vaccination_year = '不明'
                    #88 接種月（1回目） from l_ptcov19data_Column145(※右記詳細参照)/146(不明) 
                    day_vaccination_month = '不明'
                    #89 接種日（1回目） from l_ptcov19data_Column145(※右記詳細参照)/146(不明) 
                    day_vaccination_day = '不明'
                else:
                    #87 接種年（1回目） from l_ptcov19data_Column145(※右記詳細参照)/146(不明) YYYY/M/D→YYYY/MM/DD
                    day_vaccination_year = str_vaccinationdate[0]
                    #88 接種月（1回目） from l_ptcov19data_Column145(※右記詳細参照)/146(不明) 
                    day_vaccination_month = str_vaccinationdate[1]
                    #89 接種日（1回目） from l_ptcov19data_Column145(※右記詳細参照)/146(不明) 
                    day_vaccination_day = str_vaccinationdate[2]

            if vaccine_2nd == '有':
                count_vaccines += 1

                #91 新型コロナウイルスワクチン接種時の年齢（2回目） from l_ptcov19data_Column148 ※147(有)のみ
                #vaccine_2nd_age = row[147]
                #92 ワクチンの種類（2回目） from l_ptcov19data_Column151 mRNA/ウイルスベクター/不明/その他
                vaccine_type = row[150] if pd.isnull(row[150]) == False else '不明'
                #93 ワクチンの種類（その他）（2回目） from l_ptcov19data_Column※151”その他” その他
                if vaccine_type == 'その他':
                    vaccine_others = 'その他'
                    #95 製造会社（その他）（2回目） from l_ptcov19data_Column※151”その他” その他
                    vaccine_commpany = 'その他'
                else:
                    #94 製造会社（2回目） from l_ptcov19data_Column152 ファイザー/モデルナ/アストラゼネカ/不明/その他
                    vaccine_company = row[151] if pd.isnull(row[151]) == False else '不明'

                check_2ndtdate = lambda x :'不明' if pd.isnull(x) else x
                str_vaccinationdate = get_vaccinationdate(check_2ndtdate(row[153]))
                if row[154] == 1:
                    #96 接種年（2回目） from l_ptcov19data_Column154(※右記詳細参照)/155(不明) YYYY/M/D→YYYY/MM/DD
                    day_vaccination_year = '不明'
                    #97 接種月（2回目） from l_ptcov19data_Column154(※右記詳細参照)/155(不明) 
                    day_vaccination_month = '不明'
                    #98 接種日（2回目） from l_ptcov19data_Column154(※右記詳細参照)/155(不明) 
                    day_vaccination_day = '不明'
                else:
                    #96 接種年（2回目） from l_ptcov19data_Column154(※右記詳細参照)/155(不明) YYYY/M/D→YYYY/MM/DD
                    day_vaccination_year = str_vaccinationdate[0]
                    #97 接種月（2回目） from l_ptcov19data_Column154(※右記詳細参照)/155(不明) 
                    day_vaccination_month = str_vaccinationdate[1]
                    #98 接種日（2回目） from l_ptcov19data_Column154(※右記詳細参照)/155(不明) 
                    day_vaccination_day = str_vaccinationdate[2]

            if vaccine_3rd == '有':
                count_vaccines += 1

                #100 新型コロナウイルスワクチン接種時の年齢（3回目） from l_ptcov19data_Column183 182(有)のみ
                #vaccine_age = row[182]
                #101 ワクチンの種類（3回目） from l_ptcov19data_Column186 mRNA/ウイルスベクター/不明/その他
                vaccine_type = row[185] if pd.isnull(row[185]) == False else '不明'
                #102 ワクチンの種類（その他）（3回目） from l_ptcov19data_Column※186”その他” ”その他”
                if vaccine_type == 'その他':
                    vaccine_others = 'その他'
                    #104 製造会社（その他）（3回目） from l_ptcov19data_Column※186”その他” ”その他”
                    vaccine_company = 'その他'
                else:
                    #103 製造会社（3回目） from l_ptcov19data_Column187/188(不明) ファイザー/モデルナ/アストラゼネカ/不明/その他
                    if row[187] == 1:
                        vaccine_company = '不明'
                    else:
                        vaccine_company = row[186]

                check_3rddate = lambda x :'不明' if pd.isnull(x) else x
                str_vaccinationdate = get_vaccinationdate(check_3rddate(row[188]))
                if row[189] == 1:
                    #105 接種年（3回目） from l_ptcov19data_Column189(※右記詳細参照)/190(不明) YYYY/M/D→YYYY/MM/DD
                    day_vaccination_year = '不明'
                    #97 接種月（3回目） from l_ptcov19data_Column189(※右記詳細参照)/190(不明) 
                    day_vaccination_month = '不明'
                    #98 接種日（3回目） from l_ptcov19data_Column189(※右記詳細参照)/190(不明) 
                    day_vaccination_day = '不明'
                else:
                    #105 接種年（3回目） from l_ptcov19data_Column189(※右記詳細参照)/190(不明) YYYY/M/D→YYYY/MM/DD
                    day_vaccination_year = str_vaccinationdate[0]
                    #97 接種月（3回目） from l_ptcov19data_Column189(※右記詳細参照)/190(不明) 
                    day_vaccination_month = str_vaccinationdate[1]
                    #98 接種日（3回目） from l_ptcov19data_Column189(※右記詳細参照)/190(不明) 
                    day_vaccination_day = str_vaccinationdate[2]
        except Exception as e:
            cnt_person_err += 1
            cnt_err += 1
            custom_logger.warning(str(cnt_line) + '人目　' + ptname_sei + ' ' + ptname_mei + ':「ワクチン接種の有無、種類、会社、接種年月日」の入力内容を確認してください。')
            custom_logger.exception('「ワクチン接種の有無、種類、会社、接種年月日」の変換処理で予期せぬエラーが発生しました。')
            day_vaccination_year = '※不明'
            day_vaccination_month = '※不明'
            day_vaccination_day = '※不明'
            vaccine_type = '※不明'
            vaccine_company = '※不明'
            pass

        #36←110 重症化のリスク因子となる疾患の有無 from l_ptcov19data_Column156(有)/157(無) 1→"[左記文言]"
        check_severerisk = lambda x, y : '有' if x == 1 else '無' if y == 1 else '不明'
        severerisk = check_severerisk(row[155], row[156])

        #37←111 悪性腫瘍 from l_ptcov19data_Column158 1→""有"" #0→""無"""
        check_malignanttumor = lambda x : '有' if x == 1 else '無'
        malignanttumor = check_malignanttumor(row[157])

        #38←112 慢性閉塞性肺疾患（COPD） from l_ptcov19data_Column159 1→""有"" #0→""無"""
        check_copd = lambda x : '有' if x == 1 else '無'
        copd = check_copd(row[158])

        #39←113 慢性腎臓病 from l_ptcov19data_Column160 1→""有"" #0→""無"""
        check_cdk = lambda x : '有' if x == 1 else '無'
        cdk = check_cdk(row[159])

        #【NEW!項目なしのため未使用】40　重症化リスク因子となる疾患等＿心血管疾患
        #【NEW!項目なしのため未使用】41　重症化リスク因子となる疾患等＿脳血管疾患

        #42←118 喫煙歴 from l_ptcov19data_Column165 1→""有"" #0→""無"""
        check_smoker = lambda x : '有' if x == 1 else '無'
        smoker = check_smoker(row[164])

        #43←114 高血圧 from l_ptcov19data_Column161 1→""有"" #0→""無"""
        check_hbp = lambda x : '有' if x == 1 else '無'
        hbp = check_hbp(row[160])

        #44←115 糖尿病 from l_ptcov19data_Column162 1→""有"" #0→""無"""
        check_dm = lambda x : '有' if x == 1 else '無'
        dm = check_dm(row[161])

        #45←116 脂質異常症 from l_ptcov19data_Column163 1→""有"" #0→""無"""
        check_dyslipidemia = lambda x : '有' if x == 1 else '無'
        dyslipidemia = check_dyslipidemia(row[162])

        #46←117 肥満（BMI30以上） from l_ptcov19data_Column164 1→""有"" #0→""無"""
        check_bmi30ov = lambda x : '有' if x == 1 else '無'
        bmi30ov = check_bmi30ov(row[163])

        #47←120 臓器移植、免疫抑制剤、抗がん剤等の使用その他の事由により免疫の機能が低下している恐れ from l_ptcov19data_Column168(有)/169(無) 1→""[左記の通り]""
        check_immunodeficiency = lambda x, y : '有' if x == 1 else '無' if y == 1 else '無'
        immunodeficiency = check_immunodeficiency(row[167], row[168])

        #48←121 妊娠 from l_ptcov19data_Column170(有)/171(無) 170”1”→""有""/→""無"" #両方0→""無"""
        check_pregnancy = lambda x, y : '有' if x == 1 else '無' if y == 1 else '無'
        pregnancy = check_pregnancy(row[169], row[170])

        #49←119 その他 from l_ptcov19data_Column166 1→""有"" #0→""無"""
        check_otherrisk = lambda x : '有' if x == 1 else '無'
        otherrisk = check_otherrisk(row[165])

        #50←122 重症度 from l_ptcov19data_Column172"軽症"/173"中等症Ⅰ"/174"中等症Ⅱ"/175"重症" 一番重いもの該当あり
        check_severity = lambda w,x,y,z : '軽症' if w == 1 else '中等症Ⅰ' if x == 1 else '中等症Ⅱ' if y == 1  else '重症' if z == 1 else ''
        severity = check_severity(row[171], row[172], row[173], row[174])



        #※未使用43 - 検査方法（１）--現行は使ってないけどずれてるだけ？（２）に記載
        #※未使用44 - 検体（１）--現行は使ってないけどずれてるだけ？（２）に記載
        #※未使用45 - 検体採取日（１）--現行は使ってないけどずれてるだけ？（２）に記載
        #※未使用46 - 結果（１）--現行は使ってないけどずれてるだけ？（２）に記載
        #※未使用51 - 検査方法（３）
        #※未使用52 - 検体（３）
        #※未使用53 - 検体採取日（３）
        #※未使用54 - 結果（３）
        #※未使用55 - 検査方法（４）
        #※未使用56 - 検体（４）
        #※未使用57 - 検体採取日（４）
        #※未使用58 - 結果（４）
        #51←123 入院の必要性 from l_ptcov19data_Column176(有)/177(無) 
        check_needhospitalization = lambda x, y : '有' if x == 1 else '無' if y == 1 else '無'
        needhospitalization = check_needhospitalization(row[175], row[176])

        #52←108 届出時点の入院の有無 from l_ptcov19data_Column121(有)/122(無) 1→"[左記文言]"
        check_hospitalization = lambda x, y : '有' if x == 1 else '無' if y == 1 else '不明'
        hospitalization = check_hospitalization(row[120], row[121])

        #53←109 入院例のみ（入院年月日) from l_ptcov19data_Column※121”１”のみ
        if hospitalization == '有':
            try:
                #123 YYYY/M/D→YYYY/MM/DD"
                check_date_hospitalization = lambda x :'※要手入力' if pd.isnull(x) else x
                date_hospitalization = check_date_hospitalization(row[122])
                if date_hospitalization != '※要手入力':
                    hospitalizationdate = conv_dateYYYYMMDD(row[122])
            except Exception as e:
                cnt_person_err += 1
                cnt_err += 1
                custom_logger.warning(str(cnt_line) + '人目　' + ptname_sei + ' ' + ptname_mei + ':「入院年月日」の入力内容を確認してください。')
                custom_logger.exception('「入院年月日」の変換処理で予期せぬエラーが発生しました。')
                hospitalizationdate = '※要確認、手入力※カルテ確認（患者ID：' + str(row[3]) + '）→入院年月日手入力'
                 
                pass

        #54←128 入院医療機関 ※123”１”のときのみ 固定値”独立行政法人国立病院機構埼玉病院”
        if needhospitalization == '有':
            hospitalname = '独立行政法人　国立病院機構　埼玉病院'

        #60←12 - 当該者職業 from l_ptcov19data_Column73
        business = row[72]
        #60→【廃止】 初診年月日 from l_ptcov19data_Column87
        #first_me_date = conv_dateYYYYMMDD(row[86])

        """61←47 - 検査方法（２） -- from l_ptcov19data_Column89 - 1→"核酸増幅法"/
                                                     88 - 1→”分離同定”/
                                                    128 - １→”抗原検査（定性）”/
                                                    129 - １→”抗原検査（定量）”
        """
        if row[88] == 1:
            kensahouhou = '核酸増幅法'
        elif row[87] == 1:
            kensahouhou = '分離同定'
        elif row[127] == 1:
            kensahouhou = '抗原検査（定性）'
        elif row[128] == 1:
            kensahouhou = '抗原検査（定量）'


        """62←48 - 検体（２） -- from l_ptcov19data_Column前項89→60（喀痰）、（※以下、１→”[左記検体名]”）
                                                        61(気道吸引液)、
                                                        62(肺胞洗浄液)、
                                                        63(咽頭拭い液)、
                                                        64(鼻腔吸引液)、
                                                        65(鼻腔拭い液)、
                                                        115(鼻咽頭拭い液)、
                                                        116(便)、
                                                        117(唾液)、
                                                        66(剖検材料)、
                                                        67(その他)＆68(その他詳細）
                                                  前項88→51（喀痰）、
                                                        52(気道吸引液)、
                                                        53(肺胞洗浄液)、
                                                        54(咽頭拭い液)、
                                                        55(鼻腔吸引液)、
                                                        56(鼻腔拭い液)、
                                                        109(鼻咽頭拭い液)、
                                                        110(便)、
                                                        111(唾液)、
                                                        57(剖検材料)、
                                                        58(その他)＆59(その他詳細)
                                                 前項128→135(鼻腔拭い液)、124(鼻咽頭拭い液)
                                                 前項129→136(鼻腔拭い液)、130(鼻咽頭拭い液)、131(唾液)

        """
        if kensahouhou == '核酸増幅法':
            #検体：喀痰
            if row[59] == 1:
                kentai = '喀痰'
            #検体：気道吸引液
            elif row[60] == 1:
                kentai = '気道吸引液'
            #検体：肺胞洗浄液
            elif row[61] == 1:
                kentai = '肺胞洗浄液'
            #検体：咽頭拭い液
            elif row[62] == 1:
                kentai = '咽頭拭い液'
            #検体：鼻腔吸引液
            elif row[63] == 1:
                kentai = '鼻腔吸引液'
            #検体：鼻腔拭い液
            elif row[64] == 1:
                kentai = '鼻腔拭い液'
            #検体：鼻咽頭拭い液
            elif row[114] == 1:
                kentai = '鼻咽頭拭い液'
            #検体：便
            elif row[115] == 1:
                kentai = '便'
            #検体：唾液
            elif row[116] == 1:
                kentai = '唾液'
            #検体：剖検材料
            elif row[65] == 1:
                kentai = '剖検材料'
            #検体：その他&その他詳細
            elif row[66] == 1:
                kentai = row[67]
        elif kensahouhou == '分離同定':
           #検体：喀痰
            if row[50] == 1:
                kentai = '喀痰'
            #検体：気道吸引液
            elif row[51] == 1:
                kentai = '気道吸引液'
            #検体：肺胞洗浄液
            elif row[52] == 1:
                kentai = '肺胞洗浄液'
            #検体：咽頭拭い液
            elif row[53] == 1:
                kentai = '咽頭拭い液'
            #検体：鼻腔吸引液
            elif row[54] == 1:
                kentai = '鼻腔吸引液'
            #検体：鼻腔拭い液
            elif row[55] == 1:
                kentai = '鼻腔拭い液'
            #検体：鼻咽頭拭い液
            elif row[108] == 1:
                kentai = '鼻咽頭拭い液'
            #検体：便
            elif row[109] == 1:
                kentai = '便'
            #検体：唾液
            elif row[110] == 1:
                kentai = '唾液'
            #検体：剖検材料
            elif row[56] == 1:
                kentai = '剖検材料'
            #検体：その他&その他詳細
            elif row[57] == 1:
                kentai = row[58]
        elif kensahouhou == '抗原検査（定性）':
            #検体：鼻腔拭い液
            if row[134] == 1:
                kentai = '鼻腔拭い液'
            #検体：鼻咽頭拭い液
            elif row[123] == 1:
                kentai = '鼻咽頭拭い液'
        elif kensahouhou == '抗原検査（定量）':
            #検体：鼻腔拭い液
            if row[135] == 1:
                kentai = '鼻腔拭い液'
            #検体：鼻咽頭拭い液
            elif row[129] == 1:
                kentai = '鼻咽頭拭い液'
            #検体：唾液
            elif row[130] == 1:
                kentai = '唾液'

        #62 感染したと推定される年月日 from l_ptcov19data_Column86 ※要形成変換YYYY/M/D→YYYY/MM/DD
        predate_infection = '' if ((row[85] == '不詳') or (row[85] == '不明')) else row[85]

        """63←50 - 結果（２） - from l_ptcov19data_Column ※１→”[左記当該結果]”
                                                 拡散増幅→119(陽性)/120(陰性)
                                                 分離同定→113(陽性)/114(陰性)
                                                 抗原定性→126(陽性)/127(陰性)
                                                 抗原定量→133(陽性)/134(陰性)
        """
        check_result_lab = lambda x, y : '陽性' if x == 1 else '陰性' if y == 1 else '※要検査結果確認'
        if kensahouhou == '核酸増幅法':
            result_lab = check_result_lab(row[118], row[119])
        elif kensahouhou == '分離同定':
            result_lab = check_result_lab(row[112], row[113])
        elif kensahouhou == '抗原検査（定性）':
            result_lab = check_result_lab(row[125], row[126])
        elif kensahouhou == '抗原検査（定量）':
            result_lab = check_result_lab(row[132], row[133])

        if result_lab == '':
            result_lab = '※要検査結果選択'

        #処理順の関係でここに置く
        """28←49 - 検体採取日（２）- from l_ptcov19data_Column ※以下、各項目要形成変換YYYY/M/D→YYYY/MM/DD
                                                    拡散増幅→118
                                                    分離同定→112
                                                    抗原定性→125
                                                    抗原定量→132
        """        
        try:
            if kensahouhou == '核酸増幅法':
                kentaisaishudate = conv_dateYYYYMMDD(row[117])
            elif kensahouhou == '分離同定':
                kentaisaishudate = conv_dateYYYYMMDD(row[111])
            elif kensahouhou == '抗原検査（定性）':
                kentaisaishudate = conv_dateYYYYMMDD(row[124])
            elif kensahouhou == '抗原検査（定量）':
                kentaisaishudate = conv_dateYYYYMMDD(row[131])
        except Exception as e:
            cnt_person_err += 1
            cnt_err += 1
            custom_logger.warning(str(cnt_line) + '人目　' + ptname_sei + ' ' + ptname_mei + ':「検体採取日」の入力内容を確認してください。')
            custom_logger.exception('「検体採取日」を変換入力する処理で予期せぬエラーが発生しました。')
            kensahouhou = '※要検体採取日確認、手修正'
            pass

        #※未使用64←59 - 自由記述欄
        #結果陰性時の”臨床診断”コメント付加対応 -2022.8.5
        if (result_lab == '陰性' and (shindanruikei == '患者(確定例)' or shindanruikei == '擬似症患者')):
            clinicaldiagnosis_comments = '臨床診断'
        elif result_lab == '' or result_lab == '※要検査結果選択':
            clinicaldiagnosis_comments = '臨床診断※臨床診断ではないときはコメント削除必要'
        else:
            clinicaldiagnosis_comments = ''

        check_infectionroute = lambda x, y : '推定' if x == 1 else '確定' if y == 1 else '※感染原因・経路要確認'
        #65←27 - 発熱 from l_ptcov19data_Column43 --１→”有”
        check_hatsunetsu = lambda x : "有" if x == 1 else "無"
        hatsunetsu = check_hatsunetsu(row[42])

        #66←28 - 咳 from l_ptcov19data_Column44 --１→”有”
        check_seki = lambda x : "有" if x == 1 else "無"
        seki = check_seki(row[43])

        #67←29 - 咳以外の急性呼吸器症状 from l_ptcov19data_Column45 --１→”有”
        check_kyuuseikokyuuki = lambda x : "有" if x == 1 else "無"
        kyuuseikokyuuki = check_kyuuseikokyuuki(row[44])

        #68←30 - 肺炎像 from l_ptcov19data_Column100 --１→”有”
        check_haienzou = lambda x : "有" if x == 1 else "無"
        haienzou = check_haienzou(row[99])

        #69←31 - 重篤な肺炎 from l_ptcov19data_Column46 --１→”有”
        cheku_zyutokuhaien = lambda x : "有" if x == 1 else "無"
        zyutokuhaien = cheku_zyutokuhaien(45)

        #70←32 - 急性呼吸窮迫症候群 from l_ptcov19data_Column102 --１→”有”
        check_ards = lambda x : "有" if x == 1 else "無"
        ards = check_ards(row[101])

        #71←33 - 多臓器不全 from l_ptcov19data_Column108 --１→”有”
        check_tazoukifuzen = lambda x : "有" if x == 1 else "無"
        tazoukifuzen = check_tazoukifuzen(row[107])

        #72←34 - 全身倦怠感 from l_ptcov19data_Column101 --１→”有”
        check_zenshinkentaikan = lambda x : "有" if x == 1 else "無"
        zenshinkentaikan = check_zenshinkentaikan(row[100])

        #73←35 - 頭痛  from l_ptcov19data_Column103 --１→”有”
        check_zutuu = lambda x : "有" if x == 1 else "無"
        zutuu = check_zutuu(row[102])

        check_shikuchoson = lambda w,x,y,z : '市' if w == 1 else '区' if x == 1 else '町' if y == 1  else '村' if z == 1 else ''
        #74←36 - 嘔気/嘔吐 form l_ptcov19data_Column104 --１→”有”
        check_ouki = lambda x : "有" if x == 1 else "無"
        ouki = check_ouki(row[103])  

        #75←37 - 下痢 form l_ptcov19data_Column105 --１→”有”
        check_geri = lambda x : "有" if x == 1 else "無"
        geri = check_geri(row[104])

        #76←38 - 結膜炎 form l_ptcov19data_Column106 --１→”有”
        check_ketsumakuen = lambda x : "有" if x == 1 else "無"
        ketsumakuen = check_ketsumakuen(row[105])

        #77←39 - 嗅覚・味覚障害 form l_ptcov19data_Column107 --１→”有”
        check_kyukakumikaku = lambda x : "有" if x == 1 else "無"
        kyukakumikaku = check_kyukakumikaku(row[106])

        #NEW！78　症状＿咽頭痛
        #yahgee側で項目未定義のため、未実装
        #pharyngalgia

        #79←40 - 酸素飽和度（室内気）from l_ptcov19data_Column137
        spo2 = row[136]
        """
        if row[136] is None:
            spo2 = row[136]
        else:
            spo2 = mj.han_to_zen(str(row[136]))
        """
        #80←41 - その他 from l_ptcov19data_Column49

        #咽頭痛項目追加対応
        try:
            if pd.isna(row[48]) == False:
                others = row[48]
                flg_pharyngalgia = '咽頭痛' in others       
            else:
                flg_pharyngalgia = False

            if flg_pharyngalgia == True:
                pharyngalgia = '有'
                others = others.replace('咽頭痛','')
                if len(others) > 0:
                    others_1st_string = others[0]
                    ##咽頭痛抜出し後、文字列補正処理
                    if others_1st_string == '、':
                        others = others[1:]
                    others = others.replace('、、','、')
            else:
                pharyngalgia = '無'
        except Exception as e:
            cnt_person_err += 1
            cnt_err += 1
            custom_logger.warning(str(cnt_line) + '人目　' + ptname_sei + ' ' + ptname_mei + ':「咽頭痛」の入力内容を確認してください。')
            custom_logger.exception('「咽頭痛」を検知する処理で予期せぬエラーが発生しました。')
            pharyngalgia = '※無'
            pass


        #【項目なしのため未使用】81←131　ゲノム解析＿変異株PCR＿N501Y
        #【項目なしのため未使用】82←132　ゲノム解析＿変異株PCR＿L452R
        #【項目なしのため未使用】83←133　ゲノム解析＿ゲノム解析結果
        #【項目なしのため未使用】84←134　ゲノム解析＿ゲノム解析結果（ドロップダウン）
        #【項目なしのため未使用】85←135　ゲノム解析＿ゲノム解析結果（その他）
        #【項目なしのため未使用】86←136　ゲノム解析＿特記事項欄
        #【NEW!※項目なしのため未使用】87←130 感染経路分析＿リンクの有無

        #88←65 感染原因・感染経路 from l_ptcov19data_Column18(推定)/17(確定)
        infectionroute = check_infectionroute(row[17], row[16])

        #89←66 飛沫・飛沫核感染 from l_ptcov19data_Column19 - 1→"○"
        if row[18] == 1:
            himatukakukansen = "○"

        #90←67 記述欄 from l_ptcov19data_Column22
        if pd.isnull(row[21]) == False:
            desc_himatukakukansen = row[21]
        check_vaccine = lambda x, y, z : '有' if x == 1 else '無' if y == 1 else '不明' if z == 1 else ''

        #91←68 接触感染 from l_ptcov19data_Column20 - 1→"○"
        if row[19] == 1:
            sesshokukansen = "○"
            check_link = 'リンクあり'
        else:
            check_link = 'リンクなし'

        #92←69 記述欄 from l_ptcov19data_Column23
        if pd.isnull(row[22]) == False:
            desc_sesshokukansen = row[22]

        #93←70 その他 from l_ptcov19data_Column21 - 1→"○"
        if row[20] == 1:
            kansenothers = "○"

        #94←71 記述欄 from l_ptcov19data_Column90
        #if math.isnan(row[89]) == False:
        if row[89] == '':
            desc_kansenothers = row[89]

        #95←72 感染地域 from l_ptcov19data_Column24(確定)/25(推定) - 1→"[左記文言]"
        infectionarea = check_infectionroute(row[24], row[23])

        #96←73 日本国内（都道府県）from l_ptcov19data_Column26 ※※？都道府県名を正式なものにしないといけない？？
        if pd.isnull(row[25]) == False:
            infection_todofuken = search_todofuken(row[25])

        #97←74 日本国内（市区町村）from l_ptcov19data_Column31＋32(市)/33(区)/34(町)/35(村)
        if pd.isnull(row[30]) == False:
            infection_shichouson = row[30] + check_shikuchoson(row[31], row[32], row[33], row[34])

        #※未使用98←75 場所区分
        #※未使用99←76 場所名称
        #100←77 国外 from l_ptcov19data_Column99 (※※99より定義国名選択)
        if pd.isnull(row[98]) == False:
            destinationcountry = search_country(row[98])

        #101←78 詳細地域 from l_ptcov19data_Column36
        if pd.isnull(row[35]) == False:
            detailedarea = row[35]

        #102←79 渡航期間( 出国日 ) from l_ptcov19data_Column69 YYYY/M/D→YYYY/MM/DD
        if pd.isnull(row[68]) == False:
            departure_date = conv_dateYYYYMMDD(row[68])

        #103←80 渡航期間( 入国日 ) from l_ptcov19data_Column70 YYYY/M/D→YYYY/MM/DD
        if pd.isnull(row[69]) == False:
            entryinto_date = conv_dateYYYYMMDD(row[69])


        #124 新型コロナウイルス感染症の拡大に際しての時限的・特例的な取扱いによる電話や情報通信機器を用いた診療の有無 from l_ptcov19data_Column178(有)/179(無) 
        #check_telemedicine = lambda x, y : '有' if x == 1 else '無' if y == 1 else '無'
        #telemedicine = check_telemedicine(row[177], row[178])

        #※未使用125 医師が必要と認める事項 
        #※未使用104←129 パスポート番号
        #※未使用130 リンクの有無
        #※未使用137 オリンピック・パラリンピック 
        #※未使用138 オリンピック・パラリンピックの属性 
        #※未使用139 自由記述欄（担当保健所専用） 
        #※未使用140 メール送信 
        #※未使用141 My HER-SYS URL 通知 
        #※未使用142 HER-SYS ID 通知   
        #110←143 健康観察期間 ※０日で固定
        periodhealthobservation = '0'
        
        ###臨床診断対応 - 2022.8.16 Add
        ###無症状　かつ　類型：疑似症　→　臨床診断
        if shindanruikei == '擬似症患者' and all(
            hatsunetsu == '無',
            seki == '無',
            kyuuseikokyuuki == '無',
            haienzou == '無',
            zyutokuhaien == '無',
            ards == '無',
            tazoukifuzen == '無',
            zenshinkentaikan == '無',
            zutuu == '無',
            ouki == '無',
            geri == '無',
            ketsumakuen == '無',
            kyukakumikaku == '無',
            pharyngalgia == '無'
        ):
            #63	診断方法＿検査結果
            result_lab = 'その他'
            #64	診断方法＿自由記述欄
            clinicaldiagnosis_comments = '臨床診断'
            #発症日
            onset_date = ''
        else:
            result_lab = ''
            clinicaldiagnosis_comments = ''

        ####ここから行追加処理
        list_additionaldata = [[
            todaysdate, #1	報告年月日
            hpname, #2	従事する病院・診療所の名称
            drname_sei, #3	医師の氏名＿姓
            drname_mei,#4	医師の氏名＿名
            shindanruikei, #5	診断（検案）した者（死体）の類型
            ptname_sei, #6	当該者氏名＿姓
            ptname_mei, #7	当該者氏名＿名
            ptkananame_sei, #8	当該者氏名＿姓＿フリガナ
            ptkananame_mei, #9	当該者氏名＿名＿フリガナ
            gender, # 10	性別
            birthdate_YYYYMMDD,#11	生年月日
            over65yearsold,#22/11/16Add 65歳以上
            nowpostcode, #12	当該者所在地＿郵便番号
            nowtodofuken, #13	当該者所在地＿都道府県
            nowaddress1, #14	当該者所在地＿市区町村
            nowaddress2, #15	当該者所在地＿丁目番地
            nowphonenumber, #16	電話番号1
            publichealthcenter_todofuken, #17	担当保健所＿都道府県
            publichealthcenter_centername, #18	担当保健所＿保健所名称
            postcode, #19	当該者住所＿郵便番号
            todofuken, #20	当該者住所＿都道府県
            address1, #21	当該者住所＿市区町村
            address2, #22	当該者住所＿丁目番地
            phonenumber, #23	電話番号2
            guardianname_sei, #24	保護者氏名＿姓
            guardianname_mei, #25	保護者氏名＿名
            gurdianphonenumber, #26	保護者電話番号
            diagnosis_date, #27	診断（検案）年月日
            kentaisaishudate, #28	検体採取日
            onset_date, #29	発病年月日（有症状の場合）
            death_date, #30	死亡年月日（死亡者検案の場合）
            count_vaccines, #31	"新型コロナウイルスワクチン接種歴＿直近の新型コロナウイルスワクチン接種＿接種回数"
            day_vaccination_year, #32	"新型コロナウイルスワクチン接種歴＿直近の新型コロナウイルスワクチン接種＿接種年"
            day_vaccination_month, #33	"新型コロナウイルスワクチン接種歴＿直近の新型コロナウイルスワクチン接種＿接種月"
            day_vaccination_day, #34	"新型コロナウイルスワクチン接種歴＿直近の新型コロナウイルスワクチン接種＿接種日"
            vaccine_company, #35	"新型コロナウイルスワクチン接種歴＿直近の新型コロナウイルスワクチン接種＿ワクチン"
            severerisk, #36	重症化リスク因子となる疾患等の有無
            malignanttumor, #37	重症化リスク因子となる疾患等＿悪性腫瘍
            copd, #38	重症化リスク因子となる疾患等＿慢性呼吸器疾患(COPD等)
            cdk, #39	重症化リスク因子となる疾患等＿慢性腎臓病
            '無', #40	重症化リスク因子となる疾患等＿心血管疾患
            '無', #41	重症化リスク因子となる疾患等＿脳血管疾患
            smoker, #42	重症化リスク因子となる疾患等＿喫煙歴
            hbp, #43	重症化リスク因子となる疾患等＿高血圧
            dm, #44	重症化リスク因子となる疾患等＿糖尿病
            dyslipidemia, #45	重症化リスク因子となる疾患等＿脂質異常症
            bmi30ov, #46	重症化リスク因子となる疾患等＿肥満（BMI30以上）
            immunodeficiency, #47	重症化リスク因子となる疾患等＿臓器の移植、免疫抑制剤、抗がん剤等の使用、その他の事由による免疫機能の低下
            pregnancy, #48	重症化リスク因子となる疾患等＿妊娠
            otherrisk, #49	重症化リスク因子となる疾患等＿その他
            necessity_administration_TD, #22/11/16Add 治療薬の投与の必要性の有無
            necessity_administration_O2, #22/11/16Add 酸素投与の必要性の有無
            severity, #50	発生届出時点の重症度
            needhospitalization, #51	発生届出時点の重症度＿発生届出時点の入院の必要性
            hospitalization, #52	発生届出時点の重症度＿発生届出時点の入院の有無
            hospitalizationdate, #53	発生届出時点の入院＿入院年月日
            hospitalname, #54	発生届出時点の入院＿入院医療機関
            '', #55	その他の住所＿郵便番号
            '', #56	その他の住所＿都道府県
            '', #57	その他の住所＿市区町村
            '', #58	その他の住所＿丁目番地
            '', #59	電話番号3
            business, #60	職業
            kensahouhou, #61	診断方法＿検査方法
            kentai, #62	診断方法＿検体
            result_lab, #63	診断方法＿検査結果
            clinicaldiagnosis_comments, #64	診断方法＿自由記述欄
            hatsunetsu, #65	症状＿発熱
            seki, #66	症状＿咳
            kyuuseikokyuuki, #67	症状＿咳以外の急性呼吸器症状
            haienzou, #68	症状＿肺炎像
            zyutokuhaien, #69	症状＿重篤な肺炎
            ards, #70	症状＿急性呼吸窮迫症候群
            tazoukifuzen, #71	症状＿多臓器不全
            zenshinkentaikan, #72	症状＿全身倦怠感
            zutuu, #73	症状＿頭痛
            ouki, #74	症状＿嘔気/嘔吐
            geri, #75	症状＿下痢
            ketsumakuen, #76	症状＿結膜炎
            kyukakumikaku, #77	症状＿嗅覚・味覚障害
            pharyngalgia, #78	症状＿咽頭痛
            spo2, #79	症状＿酸素飽和度（室内気）
            others, #80	症状＿その他症状
            '未実施', #81	ゲノム解析＿変異株PCR＿N501Y
            '未実施', #82	ゲノム解析＿変異株PCR＿L452R
            '', #83	ゲノム解析＿ゲノム解析結果
            '', #84	ゲノム解析＿ゲノム解析結果（ドロップダウン）
            '', #85	ゲノム解析＿ゲノム解析結果（その他）
            '', #86	ゲノム解析＿特記事項欄
            '', #87	感染経路分析＿リンクの有無
            infectionarea, #95	感染経路分析＿感染場所
            infection_todofuken, #96	感染経路分析＿感染場所＿日本国内（都道府県）
            infection_shichouson, #97	感染経路分析＿感染場所＿日本国内（市区町村）
            '', #98	感染経路分析＿感染場所＿日本国内＿場所区分
            '', #99	感染経路分析＿感染場所＿日本国内＿場所名称
            destinationcountry, #100	感染経路分析＿感染場所＿国外_国
            detailedarea, #101	感染経路分析＿感染場所＿国外＿詳細地域
            departure_date, #102	感染経路分析＿感染場所＿国外＿渡航期間（出国日）
            entryinto_date, #103	感染経路分析＿感染場所＿国外＿渡航期間（入国日）
            infectionroute, #88	感染経路分析＿感染原因・感染経路
            himatukakukansen, #89	感染経路分析＿飛沫・飛沫核感染
            desc_himatukakukansen, #90	感染経路分析＿飛沫・飛沫核感染＿コメント
            sesshokukansen, #91	感染経路分析＿接触感染
            desc_sesshokukansen, #92	感染経路分析＿接触感染＿コメント
            kansenothers, #93	感染経路分析＿その他
            desc_kansenothers, #94	感染経路分析＿その他＿コメント
            '', #104	感染経路分析＿パスポート番号
            '', #105	感染経路分析＿医師が必要と認める事項
            '', #106	自由記述欄（担当保健所専用）
            '無', #107	通知設定＿メール送信
            '無',#108	通知設定＿My HER-SYS URL 通知
            '無',#109	通知設定＿HER-SYS ID 通知
            periodhealthobservation#110	健康観察期間
            ]]

    #i_input_trans_df_index += 1
    add_df = pd.DataFrame(list_additionaldata, columns=OUTPUT_COLUMNS)

    #DataFrame（結果格納用）に当該行分を追加
    result_df = pd.concat([result_df,add_df], axis=0,ignore_index=True)
    
    custom_logger.info(str(cnt_line) + '人目('+str(ptname_sei)+' '+str(ptname_mei) + ')：変換処理を終了しました。')
    custom_logger.info(str(cnt_line) + '人目('+str(ptname_sei)+' '+str(ptname_mei) + ')：エラー発生件数は' + str(cnt_person_err) + '件でした。')
    """
    except Exception as e:
        custom_logger.warning(str(cnt_line) + '人目　' + ptname_sei + ' ' + ptname_mei + ':予期せぬエラーのため、データ変換処理に失敗しました。当該者はデータに反映されません。')
        custom_logger.exception('※※※必ずHER-SYSへの全体データ手入力が必要です！※※※')
        pass
    #cnt_line++
    """
    cnt_line += 1
####全行繰り返し終了
cnt_line -= 1
custom_logger.info('【変換処理件数】'+str(cnt_line) + '人')
custom_logger.info('【エラー発生件数】'+str(cnt_err) + '件')
custom_logger.info('-'*10 + '【END】' + '-'*10)

#ERROR総件数判定
if cnt_err > 0:
    msgbox_return = messagebox.askyesno('エラー発生','データ変換時にエラーが発生しました。\nログファイル、XLXSファイルのセル色付き箇所を確認してください。\nログファイルを表示しますか？')
    if msgbox_return == 'True':
        notepad = r'C:\WINDOWS\system32\notepad.exe'
        sp.Popen([notepad, LOGFILENAME])

#OUTPUT_XLSX
try:
    custom_logger.info('-'*10 + '【XLSXファイル生成処理-START】' + '-'*10)
    #処理結果XLSX書き出し
    make_xlsx_output_exchanged_data(todaysnowdatetime, XLSXTEMPLATEFILE_FULLPATH, OUTPUT_XLSXFILE_FOLDERPATH, result_df)
except Exception as e:
    custom_logger.exception('データ変換結果のXLSXファイル書き出し処理で予期せぬエラーが発生しました。')
    messagebox.showerror('確認', 'XLSXファイルへの結果書き出し処理で予期せぬエラーが発生しました。\nExcelファイルを表示中の場合は、ファイルを閉じてからexeを再実行してください。')
    custom_logger.info('-'*10 + '【XLSXファイル生成処理-異常終了】' + '-'*10)   
    sys.exit('Make HER-SYS Input XLSX is output xlsx-file ERROR.')
else:
    xlsxfile_fullpath = OUTPUT_XLSXFILE_FOLDERPATH + 'dataexchangeandpasted_' + todaysnowdatetime + '.xlsx'
    if os.path.exists(xlsxfile_fullpath) == True:
        sp.Popen(['start', xlsxfile_fullpath], shell=True)
        custom_logger.info('【XLSXファイル生成処理】XLSXファイルは正常に出力されました')    
        custom_logger.info('-'*10 + '【XLSXファイル生成処理-END】' + '-'*10)
    else:
        custom_logger.warning('【XLSXファイル生成処理】XLSXファイルが出力されているか確認してください')    
        custom_logger.info('-'*10 + '【XLSXファイル生成処理-END】' + '-'*10)

#FILE MOVE ORG_CSV
try:
    custom_logger.info('-'*10 + '【ORG_CSVファイル移動処理-START】' + '-'*10)
    check_copyto_org_csvfile = os.path.join(MOVE_CSVFILE_FOLDERPATH, os.path.basename(org_csvfile_fullpath))
    if os.path.exists(check_copyto_org_csvfile):
        rename_org_csvfile = duplicate_rename(check_copyto_org_csvfile)
        new_org_csvfile = shutil.move(org_csvfile_fullpath, rename_org_csvfile)
        custom_logger.warning('【ORG_CSVファイル移動処理】CSV移動先フォルダに同じ名前のファイルが存在します。')    
        custom_logger.info('【ORG_CSVファイル移動処理】名前を変更してファイルを移動します。')
    else:
        new_org_csvfile = shutil.move(org_csvfile_fullpath, MOVE_CSVFILE_FOLDERPATH)
        
except Exception as e:
    custom_logger.exception('オリジナルCSVファイル移動処理で予期せぬエラーが発生しました。')
    messagebox.showerror('確認', 'CSVファイル移動処理でエラーが発生しました。\nこのエラーはCSVファイルを開いたままの状態でexeを実行すると表示される場合があります。\n\nCSVファイルを手動で移動させてください。')
    custom_logger.info('-'*10 + '【ORG_CSVファイル移動処理-異常終了】' + '-'*10)
    clear_logger_handlers(custom_logger)
    sys.exit('Make HER-SYS Input XLSX is move org-csvfile ERROR.')
    
else:
    custom_logger.info(org_csvfile_fullpath + '→' +new_org_csvfile + '【移動処理正常終了】')
    custom_logger.info('-'*10 + '【ORG_CSVファイル移動処理-END】' + '-'*10)
    clear_logger_handlers(custom_logger)
    sys.exit(0)


# 以上ですべて終了です。

In [ ]:
result_df